In [1]:

# 필요라이브러리 로딩
import pymysql
import numpy as np
import pandas as pd
import seaborn as sns  # 시각화 라이브러리, matplotlib보다 단순함
import matplotlib.pyplot as plt
import plotly.io as pio
import plotly.express as px 
plt.rcParams['font.family'] = 'Malgun Gothic'
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.max_rows', 200)
import pandas as pd 
import chardet
import sys
import matplotlib
from argparse import Namespace
import os
from tqdm import tqdm
from tqdm.autonotebook import tqdm
tqdm.pandas()

In [175]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import os
import plotly.io as pio
pio.templates.default = "plotly_white"
import pandas as pd
from pathlib import Path
from tqdm.autonotebook import tqdm
from IPython.display import display, HTML

In [20]:
# 로컬 라이브러리 
from hotel import final_hotel_data_processor
from room  import load_room_data
hotel= final_hotel_data_processor()
room= load_room_data()

In [21]:
# 버젼확인 
print("Python 버전:", sys.version)
print("NumPy 버전:", np.__version__)
print("Pandas 버전:", pd.__version__)
print("Matplotlib 버전:", matplotlib.__version__)
print("Seaborn 버전:", sns.__version__)

Python 버전: 3.9.19 (main, Mar 21 2024, 17:21:27) [MSC v.1916 64 bit (AMD64)]
NumPy 버전: 1.26.4
Pandas 버전: 2.2.2
Matplotlib 버전: 3.8.4
Seaborn 버전: 0.12.2


In [3]:
args = Namespace(
    raw_data= './raw_data/',
    output_dir='./raw_data/block/',
    output_price_processed = './raw_data/price_procssed/',
    output_price_processed_missing = './raw_data/price_procssed_missing/',
    output_price_processed_missing_date = './raw_data/price_processed_missing_date/'
    
)

# 필요 로컬함수 정의

## 리드타임내 결측값 처리

In [6]:
# 각 그룹 리드타임별 결측값 생성 함수 
def ensure_full_lead_times(group):
    existing_lead_times = group['lead_time'].unique()
    missing_lead_times = np.setdiff1d(all_lead_times, existing_lead_times)
    # 누락된 행을 위한 DataFrame 생성
    missing_rows = []
    for lead_time in missing_lead_times:
        missing_rows.append({
            'room_id': group['room_id'].iloc[0],
            'hotel_id': group['hotel_id'].iloc[0],
            'hotel_name': group['hotel_name'].iloc[0],
            'room_name': group['room_name'].iloc[0],
            'ota_type': group['ota_type'].iloc[0],
            'booking_date': group['booking_date'].iloc[0],  # booking_date를 복사
            'lead_time': lead_time,
            'high_demand': group['high_demand'].iloc[0],  # 기존 값 복사
            'dateName': group['dateName'].iloc[0],  # 기존 값 복사
            'day_of_week': group['day_of_week'].iloc[0],  # 기존 값 복사
            'remain': 1,
            'price': 0
        })
    if missing_rows:  # 누락된 행이 있을 경우에만 추가
        group = pd.concat([group, pd.DataFrame(missing_rows)], ignore_index=True)
    return group

In [266]:
def process_hotel_data(result_df):
    """
    호텔 데이터를 처리하여 가격 정보를 다루는 피벗 테이블 형식으로 변환합니다.
    
    Args:
    result_df (pd.DataFrame): 원본 데이터프레임
    
    Returns:
    pd.DataFrame: 변형된 데이터프레임
    """
    # 인덱스 설정
    result_df.set_index(['room_id', 'booking_date'], inplace=True)

    # 피벗 테이블 생성
    pivot_df = result_df.pivot_table(index=['room_id', 'booking_date'], columns='lead_time', values='price', fill_value=0)
    pivot_df.reset_index(inplace=True)

    # 메타 데이터 추출 및 중복 제거
    result_meta = result_df.drop(columns=['price', 'lead_time', 'remain', 'date'])
    result_meta.reset_index(inplace=True)
    result_meta = result_meta.drop_duplicates()

    # 데이터 병합
    result_final = result_meta.merge(pivot_df, on=['room_id', 'booking_date'], how='left')
    result_final.replace(0, np.nan, inplace=True)
    # result_final = result_final.drop([30, 31], axis=1)  # 특정 열 제거

    # 가격 정보 열에 대해서만 fillna 적용
    price_columns = [i for i in range(32)] 
    result_final[price_columns] = result_final[price_columns].apply(
        lambda row: row.fillna(method='bfill').fillna(method='ffill'), axis=1)

    # 열 이름 변경
    result_final.columns = ['price_' + str(int(col)) if isinstance(col, float) and col.is_integer() else col 
                            for col in result_final.columns]

    return result_final

In [8]:
def save_hotel_data(df, output_path, encoding='utf-8-sig'):
    # 호텔데이터 저장 함수
    if df.shape[0] == 0:
        pass
    # 호텔 이름을 데이터프레임에서 추출
    else:
        hotel_name = df['hotel_name'].unique()[0]

        # 방 번호의 고유 개수를 계산
        room_n = df['room_id'].nunique()

        # 파일 이름 설정
        file_name = f"{output_path}{hotel_name}_{room_n}.csv"

        # CSV 파일로 저장
        df.to_csv(file_name, index=False, encoding=encoding)

## 북킹데이트별 결측값 처리

In [267]:
def missing_processor(df,holiday):
    # 공휴일 데이터 로드
    # df와 holiday의 booking_date 칼럼을 datetime 타입으로 변환합니다.
    df['booking_date'] = pd.to_datetime(df['booking_date'])
    holiday.rename(columns={'date':'booking_date'}, inplace=True)
    holiday['booking_date'] = pd.to_datetime(holiday['booking_date'])

    # df의 모든 고유 조합을 추출합니다.
    unique_combinations = df[['room_id', 'hotel_id', 'hotel_name', 'ota_type', 'room_name']].drop_duplicates()

    # 모든 날짜와 고유 조합을 크로스 조인합니다.
    all_dates = pd.DataFrame(holiday['booking_date'].unique(), columns=['booking_date'])
    cross_joined = unique_combinations.assign(key=1).merge(all_dates.assign(key=1), on='key').drop('key', axis=1)

    # cross_joined와 holiday를 booking_date에 대해 외부 조인합니다.
    full_data = cross_joined.merge(holiday, on='booking_date', how='left')

    # full_data와 df를 병합합니다. 이 때 누락된 데이터는 NaN으로 표시됩니다.
    result = full_data.merge(df, on=['room_id', 'booking_date', 'hotel_id', 'hotel_name', 'ota_type', 'room_name', 'day_of_week', 'dateName', 'high_demand'], how='left')

    # price_0부터 price_29까지의 칼럼에 대해 결측값을 'nearest' 방법으로 보간
    for i in range(32):  # 0부터 29까지
        column_name = f'price_{i}'
        result[column_name] = result[column_name].interpolate(method='nearest')
        result[column_name] = result[column_name].bfill()
        result[column_name] = result[column_name].ffill()
    return result


In [11]:
def save_hotel_data2(df, output_path, encoding='utf-8-sig'):
    
    if df.shape[0] == 0:
        pass
    # 호텔 이름을 데이터프레임에서 추출
    else:
        hotel_name = df['hotel_name'].unique()[0]

        # 방 번호의 고유 개수를 계산
        room_n = df['room_id'].nunique()
        
        # 모든 room_id 그룹의 크기를 계산
        room_id_sizes = df.groupby('room_id').size()
        print(room_id_sizes)

        # all_dates의 길이
        all_dates_length = len(all_dates)

        # 각 room_id 그룹의 크기가 all_dates의 길이와 동일한지 확인
        assert (room_id_sizes == all_dates_length).all(), "룸아이별로충분한 시계열값이 없습니다"

        # 파일 이름 설정
        file_name = f"{output_path}{hotel_name}_{room_n}.csv"

        # CSV 파일로 저장
        df.to_csv(file_name, index=False, encoding=encoding)

# 결측값 정의

## 리드타임
* 호텔업계의 데이터를 이해하기 위해서는 반드시 '리드타임'이라는 개념이 이해되어야한다.
* 호텔업계에서 결제는 일반적으로 물건을 살때와 다르게 '선결제'라는 개념이 존재한다.
* 예를들어 나는 4월 30일에 숙박할 예정이지만 대게는 당일에 가서 구매하는게 아니라 2일전 , 3일전, 일주일전에 구매를 한다. 
* 실제 예약일에 앞서서 구매를 하는 시점을 리드타임이라고 하고 본 데이터 세트에서는 price_0(당일)을 시작으로 price_31(31일)까지 존재한다

## 리드타임과 호텔가격
* 이런 '선결제'라는 독특한 방식은 호텔 객실가격에 영향을 미친다. 즉 같은 객실이라 할지라도 당일 부터 +31까지 가격이 다를수 있다
* 이것이 다른 업계의 일반적인 시계열과 구분되는 지점이다. 아이폰은 별일없으면 한달전 가격과 오늘의 가격이 동일하다. 호텔객실은 그렇지 않다
*때문에 예약일 당 리드타임 가격이 모두 있어야 제대로 된 가격 모델예측을 할 수 있다




##  본과제에 생길수 있는 결측값의 종류
* 크게 두가지 수준의 결측치가 생길수 있다
     1. 숙박예약일 내 리드타임 수준  결측치
     2. 숙박예약일 전체에 대한 결측치




### 숙박예약일 내 리드타임 수준  결측치

In [236]:
# 샘플로드 
df=pd.read_csv(args.output_price_processed+'이비스앰배서더부산해운대_3.csv', parse_dates=False)
# 공휴일 데이터 로드
holiday=pd.read_csv(args.raw_data+"/holiday_calender_2022_2024.csv")
# 데이터 결합 
validate_df=pd.merge(df,holiday, how='inner', left_on='booking_date', right_on='date')
# 리드타임 생성
all_lead_times = np.arange(32)
# room_id도 포함하여 그룹화 및 함수 적용
result_df = validate_df.groupby(['room_id', 'hotel_name','hotel_id', 'room_name', 'ota_type', 'booking_date']).apply(ensure_full_lead_times).reset_index(drop=True)
# booking_date를 인덱스로 설정하지 않고 다중 인덱스로 처리
result_df.set_index(['room_id', 'booking_date'], inplace=True)
# pivot_table을 사용하여 room_id와 booking_date 별 리드타임에 따른 가격을 칼럼화
# 와이드 포맷 테이블 형성 -- 결측값은 0으로 채움
pivot_df = result_df.pivot_table(index=['room_id', 'booking_date'], columns='lead_time', values='price', fill_value=0)
pivot_df.reset_index(inplace=True)
#  와이이드 포맷은 가격데이터 밖에 없음 여기에 메타 데이터 추가 루프 
result_meta=result_df.drop(columns=['price','lead_time','remain','date'])
result_meta.reset_index(inplace=True)
result_meta=result_meta.drop_duplicates()
result_final=result_meta.merge(pivot_df, how='left', on=['room_id','booking_date'])
# 0 널값으로 변경 
result_final.replace(0, np.nan, inplace=True)
# D+30 , D=31 삭제 : 너무 결측값이 많음 -> 많아도 일단 채우고 없애자;;
#result_final=result_final.drop([30,31], axis=1)
result_final


,room_id,booking_date,hotel_id,hotel_name,ota_type,room_name,day_of_week,dateName,high_demand,0,...,22,23,24,25,26,27,28,29,30,31
0,37,2022-01-01,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Saturday,1월1일,Y,96800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,37,2022-01-02,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Sunday,N,N,47520,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,37,2022-01-03,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Monday,N,N,47520,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,37,2022-01-04,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Tuesday,N,N,47520,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,37,2022-01-05,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Wednesday,N,N,47520,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2341,41,2024-03-12,120,이비스앰배서더부산해운대,GOODCHOICE,스탠다드 더블,Tuesday,N,N,NaN,...,NaN,NaN,NaN,NaN,NaN,71500,71500,71500,71500,71500
2342,41,2024-03-13,120,이비스앰배서더부산해운대,GOODCHOICE,스탠다드 더블,Wednesday,N,N,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,71500,71500,71500,71500
2343,41,2024-03-14,120,이비스앰배서더부산해운대,GOODCHOICE,스탠다드 더블,Thursday,N,N,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71500,71500,71500
2344,41,2024-03-15,120,이비스앰배서더부산해운대,GOODCHOICE,스탠다드 더블,Friday,N,Y,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82500,82500


In [223]:
ts_df=result_final.query('room_id==37')
ts_df['booking_date'] = pd.to_datetime(ts_df['booking_date'])
ts_df=ts_df.set_index('booking_date')


In [226]:
ts_df.columns

Index(['room_id', 'hotel_id', 'hotel_name', 'ota_type', 'room_name',
       'day_of_week', 'dateName', 'high_demand', 'price_0', 'price_1',
       'price_2', 'price_3', 'price_4', 'price_5', 'price_6', 'price_7',
       'price_8', 'price_9', 'price_10', 'price_11', 'price_12', 'price_13',
       'price_14', 'price_15', 'price_16', 'price_17', 'price_18', 'price_19',
       'price_20', 'price_21', 'price_22', 'price_23', 'price_24', 'price_25',
       'price_26', 'price_27', 'price_28', 'price_29', 'price_30', 'price_31'],
      dtype='object')

In [160]:
result_final2=result_final.copy()

In [237]:
# 숫자형 열 이름 지정
price_columns = [i for i in range(32)]  # 0부터 15까지
# 가격 정보 열에 대해서만 fillna를 적용
result_final[price_columns] = result_final[price_columns].apply(lambda row: row.fillna(method='bfill').fillna(method='ffill'), axis=1)


In [238]:
result_final.columns = ['price_' + str(int(col)) if isinstance(col, float) and col.is_integer() else col for col in result_final.columns]
result_final

,room_id,booking_date,hotel_id,hotel_name,ota_type,room_name,day_of_week,dateName,high_demand,price_0,...,price_22,price_23,price_24,price_25,price_26,price_27,price_28,price_29,price_30,price_31
0,37,2022-01-01,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Saturday,1월1일,Y,96800,...,114400,114400,114400,114400,114400,114400,114400,114400,114400,114400
1,37,2022-01-02,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Sunday,N,N,47520,...,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520
2,37,2022-01-03,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Monday,N,N,47520,...,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520
3,37,2022-01-04,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Tuesday,N,N,47520,...,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520
4,37,2022-01-05,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Wednesday,N,N,47520,...,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2341,41,2024-03-12,120,이비스앰배서더부산해운대,GOODCHOICE,스탠다드 더블,Tuesday,N,N,71500,...,71500,71500,71500,71500,71500,71500,71500,71500,71500,71500
2342,41,2024-03-13,120,이비스앰배서더부산해운대,GOODCHOICE,스탠다드 더블,Wednesday,N,N,71500,...,71500,71500,71500,71500,71500,71500,71500,71500,71500,71500
2343,41,2024-03-14,120,이비스앰배서더부산해운대,GOODCHOICE,스탠다드 더블,Thursday,N,N,71500,...,71500,71500,71500,71500,71500,71500,71500,71500,71500,71500
2344,41,2024-03-15,120,이비스앰배서더부산해운대,GOODCHOICE,스탠다드 더블,Friday,N,Y,82500,...,82500,82500,82500,82500,82500,82500,82500,82500,82500,82500


In [172]:
from src.utils import plotting_utils
def format_plot(fig, legends = None, xlabel="Time", ylabel="Value", figsize=(500,900), font_size=15, title_font_size=20):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t:  t.update(name = next(names)))
    fig.update_layout(
            autosize=False,
            width=figsize[1],
            height=figsize[0],
            title={
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            titlefont={
                "size": 20
            },
            legend_title = None,
            legend=dict(
                font=dict(size=font_size),
                orientation="h",
                yanchor="bottom",
                y=0.98,
                xanchor="right",
                x=1,
            ),
            yaxis=dict(
                title_text=ylabel,
                titlefont=dict(size=font_size),
                tickfont=dict(size=font_size),
            ),
            xaxis=dict(
                title_text=xlabel,
                titlefont=dict(size=font_size),
                tickfont=dict(size=font_size),
            )
        )
    return fig

In [187]:
fig = px.line(ts_df, y=4, title="price_0 for room_id 37")
fig = format_plot(fig, ylabel="hotel_price")
fig

### 숙박예약일 전체에 대한 결측치

In [26]:
# 샘플로드 
df=pd.read_csv(args.output_price_processed+'이비스앰배서더부산해운대_3.csv', parse_dates=False)
# 공휴일 데이터 로드
holiday=pd.read_csv(args.raw_data+"/holiday_calender_2022_2024.csv")
# 데이터 결합 
validate_df=pd.merge(df,holiday, how='inner', left_on='booking_date', right_on='date')
# 리드타임 생성
all_lead_times = np.arange(32)
# room_id도 포함하여 그룹화 및 함수 적용
result_df = validate_df.groupby(['room_id', 'hotel_name','hotel_id', 'room_name', 'ota_type', 'booking_date']).apply(ensure_full_lead_times).reset_index(drop=True)
# booking_date를 인덱스로 설정하지 않고 다중 인덱스로 처리
result_df.set_index(['room_id', 'booking_date'], inplace=True)
# pivot_table을 사용하여 room_id와 booking_date 별 리드타임에 따른 가격을 칼럼화
# 결측값은 0으로 채움
pivot_df = result_df.pivot_table(index=['room_id', 'booking_date'], columns='lead_time', values='price', fill_value=0)
pivot_df.reset_index(inplace=True)

In [30]:
###  글로벌 booking_date 최대값 찾기 
from pathlib import Path
output_price_processed_path = Path(args.output_price_processed)

max_date = None
for f in tqdm(output_price_processed_path.glob("*.csv")):
    df = pd.read_csv(f, parse_dates=False)
    df['booking_date'] = pd.to_datetime(df['booking_date'], yearfirst=True)
    if max_date is None:
        max_date = df['booking_date'].max()
    else:
        if df['booking_date'].max()>max_date:
            max_date = df['booking_date'].max()
print(f"Max Date across all blocks: {max_date}")

0it [00:00, ?it/s]

Max Date across all blocks: 2024-03-31 00:00:00


In [36]:
###  글로벌 booking_date start_date 찾기 
from pathlib import Path
output_price_processed_path = Path(args.output_price_processed)

min_date = None
for f in tqdm(output_price_processed_path.glob("*.csv")):
    df = pd.read_csv(f, parse_dates=False)
    df['booking_date'] = pd.to_datetime(df['booking_date'], yearfirst=True)
    if min_date is None:
        min_date = df['booking_date'].min()
    else:
        if df['booking_date'].min()<min_date:
            min_date = df['booking_date'].min()
print(f"Min Date across all blocks: {min_date}")

0it [00:00, ?it/s]

Min Date across all blocks: 2022-01-01 00:00:00


In [38]:
# 시작 날짜와 종료 날짜 사이의 날짜 범위 생성
date_range = pd.date_range(start=min_date, end=max_date)
all_dates = date_range.strftime('%Y-%m-%d').tolist()
# 날짜 범위의 길이 계산 (총 날짜 수)
total_days = len(date_range)

In [193]:
ts_df2

,room_id,hotel_id,hotel_name,ota_type,room_name,day_of_week,dateName,high_demand,0,1,...,22,23,24,25,26,27,28,29,30,31
booking_date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,37,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Saturday,1월1일,Y,96800,114400,...,114400,114400,114400,114400,114400,114400,114400,114400,114400,114400
2022-01-02,37,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Sunday,N,N,47520,47520,...,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520
2022-01-03,37,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Monday,N,N,47520,47520,...,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520
2022-01-04,37,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Tuesday,N,N,47520,47520,...,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520
2022-01-05,37,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Wednesday,N,N,47520,47520,...,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-12,37,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Tuesday,N,N,57200,57200,...,57200,57200,57200,57200,57200,57200,57200,57200,57200,57200
2024-03-13,37,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Wednesday,N,N,57200,57200,...,57200,57200,57200,57200,57200,57200,57200,57200,57200,57200
2024-03-14,37,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Thursday,N,N,57200,57200,...,57200,57200,57200,57200,57200,57200,57200,57200,57200,57200


In [195]:
ts_df2.columns = ['price_' + str(int(col)) if isinstance(col, float) and col.is_integer() else col for col in ts_df2.columns]
ts_df2

,room_id,hotel_id,hotel_name,ota_type,room_name,day_of_week,dateName,high_demand,price_0,price_1,...,price_22,price_23,price_24,price_25,price_26,price_27,price_28,price_29,price_30,price_31
booking_date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,37,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Saturday,1월1일,Y,96800,114400,...,114400,114400,114400,114400,114400,114400,114400,114400,114400,114400
2022-01-02,37,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Sunday,N,N,47520,47520,...,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520
2022-01-03,37,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Monday,N,N,47520,47520,...,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520
2022-01-04,37,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Tuesday,N,N,47520,47520,...,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520
2022-01-05,37,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Wednesday,N,N,47520,47520,...,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-12,37,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Tuesday,N,N,57200,57200,...,57200,57200,57200,57200,57200,57200,57200,57200,57200,57200
2024-03-13,37,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Wednesday,N,N,57200,57200,...,57200,57200,57200,57200,57200,57200,57200,57200,57200,57200
2024-03-14,37,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Thursday,N,N,57200,57200,...,57200,57200,57200,57200,57200,57200,57200,57200,57200,57200


In [51]:
all_dates

['2022-01-01',
 '2022-01-02',
 '2022-01-03',
 '2022-01-04',
 '2022-01-05',
 '2022-01-06',
 '2022-01-07',
 '2022-01-08',
 '2022-01-09',
 '2022-01-10',
 '2022-01-11',
 '2022-01-12',
 '2022-01-13',
 '2022-01-14',
 '2022-01-15',
 '2022-01-16',
 '2022-01-17',
 '2022-01-18',
 '2022-01-19',
 '2022-01-20',
 '2022-01-21',
 '2022-01-22',
 '2022-01-23',
 '2022-01-24',
 '2022-01-25',
 '2022-01-26',
 '2022-01-27',
 '2022-01-28',
 '2022-01-29',
 '2022-01-30',
 '2022-01-31',
 '2022-02-01',
 '2022-02-02',
 '2022-02-03',
 '2022-02-04',
 '2022-02-05',
 '2022-02-06',
 '2022-02-07',
 '2022-02-08',
 '2022-02-09',
 '2022-02-10',
 '2022-02-11',
 '2022-02-12',
 '2022-02-13',
 '2022-02-14',
 '2022-02-15',
 '2022-02-16',
 '2022-02-17',
 '2022-02-18',
 '2022-02-19',
 '2022-02-20',
 '2022-02-21',
 '2022-02-22',
 '2022-02-23',
 '2022-02-24',
 '2022-02-25',
 '2022-02-26',
 '2022-02-27',
 '2022-02-28',
 '2022-03-01',
 '2022-03-02',
 '2022-03-03',
 '2022-03-04',
 '2022-03-05',
 '2022-03-06',
 '2022-03-07',
 '2022-03-

In [196]:
for i in ts_df['room_id'].unique():
    filtered_df = ts_df.query(f'room_id == {i}')
    print(f'Shape for room_id {i}:', filtered_df.shape)

Shape for room_id 37: (761, 40)


In [197]:
# 각 room_id에 대한 모든 가능한 (room_id, booking_date) 조합 생성
unique_room_ids = ts_df2['room_id'].unique()
all_combinations = pd.MultiIndex.from_product([unique_room_ids, all_dates], names=['room_id', 'booking_date'])

# 모든 조합을 DataFrame으로 변환
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

# 원래 데이터와 조합된 데이터 병합
full_df = pd.merge(all_combinations_df, ts_df2, on=['room_id', 'booking_date'], how='left')
full_df

,room_id,booking_date,hotel_id,hotel_name,ota_type,room_name,day_of_week,dateName,high_demand,price_0,...,price_22,price_23,price_24,price_25,price_26,price_27,price_28,price_29,price_30,price_31
0,37,2022-01-01,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Saturday,1월1일,Y,96800,...,114400,114400,114400,114400,114400,114400,114400,114400,114400,114400
1,37,2022-01-02,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Sunday,N,N,47520,...,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520
2,37,2022-01-03,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Monday,N,N,47520,...,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520
3,37,2022-01-04,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Tuesday,N,N,47520,...,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520
4,37,2022-01-05,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Wednesday,N,N,47520,...,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,37,2024-03-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
817,37,2024-03-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
818,37,2024-03-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
819,37,2024-03-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [206]:
full_df.price_0.min()

44000.0

### 계절성 보강으로 결측값 채우기 
* 원래 애초에 데이터값을 필터할때 결측값이 일주일 미만인것만 뽑았다고 생각했는데
* 실제 인스턴스를 보니 일주일이상으로 값이 없는 경우가 많았다. 
* 돌아가기도 귀찮고 이 경우에 대해서 계절성보강으로 채우겠다

In [210]:
from src.imputation.interpolation import SeasonalInterpolation

recovered_matrix_seas_interp_weekday_onday = SeasonalInterpolation(seasonal_period=7,
                                                     decomposition_strategy="additive", 
                                                     interpolation_strategy="spline", 
                                                     interpolation_args={"order":3}, 
                                                     min_value=57200).fit_transform(full_df.price_0.values.reshape(-1,1))

full_df['seas_interp_weekday_imputed'] = recovered_matrix_seas_interp_weekday_onday
full_df.tail(100)

,room_id,booking_date,hotel_id,hotel_name,ota_type,room_name,day_of_week,dateName,high_demand,price_0,...,price_23,price_24,price_25,price_26,price_27,price_28,price_29,price_30,price_31,seas_interp_weekday_imputed
721,37,2023-12-23,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Saturday,N,Y,132000,...,132000,132000,132000,132000,132000,132000,132000,132000,132000,132000
722,37,2023-12-24,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Sunday,N,Y,132000,...,132000,132000,132000,132000,132000,132000,132000,132000,132000,132000
723,37,2023-12-25,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Monday,기독탄신일,N,60500,...,88000,88000,88000,88000,88000,88000,88000,88000,88000,60500
724,37,2023-12-26,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Tuesday,N,N,60500,...,88000,88000,88000,88000,88000,88000,88000,88000,88000,60500
725,37,2023-12-27,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Wednesday,N,N,60500,...,88000,88000,88000,88000,88000,88000,88000,88000,88000,60500
726,37,2023-12-28,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Thursday,N,N,60500,...,88000,88000,88000,88000,88000,88000,88000,88000,88000,60500
727,37,2023-12-29,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Friday,N,Y,66000,...,132000,132000,132000,132000,132000,132000,132000,132000,132000,66000
728,37,2023-12-30,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Saturday,N,Y,138600,...,158400,158400,158400,158400,158400,158400,158400,158400,158400,138600
729,37,2023-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87305
730,37,2024-01-01,120,이비스앰배서더부산해운대,GOODCHOICE,[레이트버드-18시 체크인] 스탠다드 더블,Monday,1월1일,N,57200,...,57200,57200,57200,57200,57200,57200,57200,57200,57200,57200


In [211]:
full_df.isnull().sum()

room_id                         0
booking_date                    0
hotel_id                       60
hotel_name                     60
ota_type                       60
room_name                      60
day_of_week                    60
dateName                       60
high_demand                    60
price_0                        60
price_1                        60
price_2                        60
price_3                        60
price_4                        60
price_5                        60
price_6                        60
price_7                        60
price_8                        60
price_9                        60
price_10                       60
price_11                       60
price_12                       60
price_13                       60
price_14                       60
price_15                       60
price_16                       60
price_17                       60
price_18                       60
price_19                       60
price_20      

In [ ]:
mae_weekday_half_hour = mean_absolute_error(ts_df.loc[window, "seas_interp_weekday_half_hour_imputed"], ts_df.loc[window, "energy_consumption"])

In [98]:
for i in full_df['room_id'].unique():
    filtered_df = full_df.query(f'room_id == {i}')
    print(f'Shape for room_id {i}:', filtered_df.shape)

Shape for room_id 37: (821, 34)
Shape for room_id 38: (821, 34)
Shape for room_id 41: (821, 34)


In [61]:
df3=pd.read_csv("./raw_data/hotel_price_target_2024_01_03.csv")

In [63]:
df3.query('room_id==38')

,hotel_id,room_id,booking_date,scanned_date,remain,price,date_diff,scanned_date_date,ota_type,region
2936862,120,38,2024-01-01,2023-12-04 12:00:00,1,60775,27 days 12:00:00,2023-12-04,GOODCHOICE,부산
2936863,120,38,2024-01-01,2023-12-04 23:00:00,1,60775,27 days 01:00:00,2023-12-04,GOODCHOICE,부산
2936864,120,38,2024-01-01,2023-12-05 00:00:00,1,60775,27 days 00:00:00,2023-12-05,GOODCHOICE,부산
2936865,120,38,2024-01-01,2023-12-05 23:00:00,1,60775,26 days 01:00:00,2023-12-05,GOODCHOICE,부산
2936866,120,38,2024-01-01,2023-12-06 00:00:00,1,60775,26 days 00:00:00,2023-12-06,GOODCHOICE,부산
...,...,...,...,...,...,...,...,...,...,...
37793201,120,38,2024-03-15,2024-02-13 11:00:00,1,70125,30 days 13:00:00,2024-02-13,GOODCHOICE,부산
37793202,120,38,2024-03-15,2024-02-13 23:00:00,1,70125,30 days 01:00:00,2024-02-13,GOODCHOICE,부산
37793203,120,38,2024-03-15,2024-02-14 00:00:00,1,70125,30 days 00:00:00,2024-02-14,GOODCHOICE,부산
37793204,120,38,2024-03-15,2024-02-14 23:00:00,1,70125,29 days 01:00:00,2024-02-14,GOODCHOICE,부산


# 전체 루프

In [272]:
base_dir = args.output_price_processed
base_dir

'./raw_data/price_procssed/'

In [273]:
csv_files= [file for file in os.listdir(base_dir) if file.endswith(".csv")]
csv_files

['L7강남_11.csv',
 'L7홍대_6.csv',
 '관광호텔뮤제오_2.csv',
 '그랜드조선제주_3.csv',
 '그랜드플라자청주호텔_3.csv',
 '그랩디오션송도_16.csv',
 '글래드강남코엑스센터_4.csv',
 '글래드여의도_11.csv',
 '글래드호텔마포_2.csv',
 '글로스터제주_1.csv',
 '나인트리프리미어호텔명동2_13.csv',
 '나인트리호텔명동_10.csv',
 '노보텔앰배서더서울동대문_12.csv',
 '노보텔앰배서더서울용산_2.csv',
 '더클래스300_4.csv',
 '라마다앙코르해운대_4.csv',
 '라발스호텔부산_11.csv',
 '라한셀렉트경주_1.csv',
 '롯데리조트속초_4.csv',
 '롯데시티호텔울산_10.csv',
 '롯데호텔부산_4.csv',
 '롯데호텔서울_18.csv',
 '롯데호텔제주_14.csv',
 '마리안느호텔_5.csv',
 '목시서울인사동호텔_3.csv',
 '미포오션사이드호텔_1.csv',
 '밀리오레서울명동_3.csv',
 '베니키아프리미어호텔해운대_18.csv',
 '베스트루이스해밀턴해운대_7.csv',
 '베스트웨스턴군산_1.csv',
 '베스트웨스턴프리미어강남_3.csv',
 '베스트웨스턴해운대호텔_9.csv',
 '베이하운드호텔_4.csv',
 '부산라메르호텔_7.csv',
 '부산비즈니스호텔_4.csv',
 '서면라이온호텔부산_2.csv',
 '서울신라호텔_3.csv',
 '서울프린스호텔_20.csv',
 '세종호텔_4.csv',
 '센텀프리미어호텔_14.csv',
 '센트럴파크호텔부산_5.csv',
 '솔라리아니시테츠부산_3.csv',
 '솔라리아니시테츠호텔서울명동_5.csv',
 '스위스그랜드호텔_5.csv',
 '스위트호텔남원_3.csv',
 '스탠포드인부산_7.csv',
 '시그니엘부산_1.csv',
 '신라스테이마포_6.csv',
 '신라스테이삼성_6.csv',
 '신라스테이서대문_4.csv',
 '신라스테이서부산_9.csv',
 '신라스테이서초_1.csv'

In [270]:
# 공휴일 데이터 로드
holiday=pd.read_csv(args.raw_data+"/holiday_calender_2022_2024.csv")
# 리드타임 생성
all_lead_times = np.arange(32)

In [268]:
def save_hotel_data(df, output_path, encoding='utf-8-sig'):
    
    if df.shape[0] == 0:
        pass
    # 호텔 이름을 데이터프레임에서 추출
    else:
        hotel_name = df['hotel_name'].unique()[0]

        # 방 번호의 고유 개수를 계산
        room_n = df['room_id'].nunique()

        # 파일 이름 설정
        file_name = f"{output_path}{hotel_name}_{room_n}.csv"
        
        print(f"{output_path}{hotel_name}_{room_n}완료")

        # CSV 파일로 저장
        df.to_csv(file_name, index=False, encoding=encoding)

In [274]:
# 전체루프 돌리기 
tqdm.pandas()

for file in tqdm(csv_files):

    file_path = os.path.join(base_dir, file)
    
    # 호텔별 인스턴스 가져오기 
    df = pd.read_csv(file_path)
    
    # 데이터 결합 + 캘린더데이터 
    validate_df=pd.merge(df,holiday, how='inner', left_on='booking_date', right_on='date')
    
    # 리드타임 결측치 허구 라인 만들기
    result_df = validate_df.groupby(['room_id', 'hotel_name','hotel_id', 'room_name', 'ota_type', 'booking_date']).apply(ensure_full_lead_times).reset_index(drop=True)
    
    # 리드타임 결측치 보강이후 데이터프레임 형성 
    processed_data = process_hotel_data(result_df)
    
    
    # 처리 데이터 아웃풋
    save_hotel_data(processed_data,args.output_price_processed_missing)
    
    # print
    print('\n')

  0%|          | 0/109 [00:00<?, ?it/s]

./raw_data/price_procssed_missing/L7강남_11완료


./raw_data/price_procssed_missing/L7홍대_6완료


./raw_data/price_procssed_missing/관광호텔뮤제오_2완료


./raw_data/price_procssed_missing/그랜드조선제주_3완료


./raw_data/price_procssed_missing/그랜드플라자청주호텔_3완료


./raw_data/price_procssed_missing/그랩디오션송도_16완료


./raw_data/price_procssed_missing/글래드강남코엑스센터_4완료


./raw_data/price_procssed_missing/글래드여의도_11완료


./raw_data/price_procssed_missing/글래드호텔마포_2완료


./raw_data/price_procssed_missing/글로스터제주_1완료


./raw_data/price_procssed_missing/나인트리프리미어호텔명동2_13완료


./raw_data/price_procssed_missing/나인트리호텔명동_10완료


./raw_data/price_procssed_missing/노보텔앰배서더서울동대문_12완료


./raw_data/price_procssed_missing/노보텔앰배서더서울용산_2완료


./raw_data/price_procssed_missing/더클래스300_4완료


./raw_data/price_procssed_missing/라마다앙코르해운대_4완료


./raw_data/price_procssed_missing/라발스호텔부산_11완료


./raw_data/price_procssed_missing/라한셀렉트경주_1완료


./raw_data/price_procssed_missing/롯데리조트속초_4완료


./raw_data/price_procssed_missing/롯데시티호텔울산_10완료


./raw_data/pric

In [361]:
data_frames = []

# 각 파일을 읽고 데이터 프레임 리스트에 추가합니다.
for file in tqdm(csv_files):
    if file.endswith('.csv'):  # 확장자가 .csv인 파일만 처리
        file_path = os.path.join(base_dir, file)
        
        # 파일을 읽어 DataFrame으로 변환
        df = pd.read_csv(file_path)
        
        # DataFrame 리스트에 추가
        data_frames.append(df)

# 모든 DataFrame을 하나로 병합
combined_df = pd.concat(data_frames, ignore_index=True)

  0%|          | 0/109 [00:00<?, ?it/s]

In [366]:
combined_df

,room_id,booking_date,hotel_id,hotel_name,ota_type,room_name,day_of_week,dateName,high_demand,price_0,...,price_20,price_21,price_22,price_23,price_24,price_25,price_26,price_27,price_28,price_29
0,7854,2022-01-01,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),Saturday,1월1일,Y,123200,...,123200,123200,123200,123200,123200,123200,123200,123200,123200,123200
1,7854,2022-01-02,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),Sunday,N,N,92400,...,92400,92400,92400,92400,92400,92400,92400,92400,92400,92400
2,7854,2022-01-03,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),Monday,N,N,92400,...,92400,92400,92400,92400,92400,92400,92400,92400,92400,92400
3,7854,2022-01-04,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),Tuesday,N,N,92400,...,92400,92400,92400,92400,92400,92400,92400,92400,92400,92400
4,7854,2022-01-05,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),Wednesday,N,N,88000,...,92400,92400,92400,92400,92400,92400,92400,92400,92400,92400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480574,42358,2024-03-26,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,Tuesday,N,N,458590,...,368988,368343,349892,346064,336637,340674,338576,327063,334262,331177
480575,42358,2024-03-27,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,Wednesday,N,N,394460,...,367840,331177,329120,325006,318835,321920,321920,317806,317806,319864
480576,42358,2024-03-28,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,Thursday,N,N,595320,...,531978,531978,461890,468629,401658,417005,369585,367502,315059,291134
480577,42358,2024-03-29,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,Friday,N,Y,547792,...,547792,547792,547792,547792,547792,547792,547792,547792,548080,547849


In [364]:
combined_df.room_name.nunique()

382

In [365]:
combined_df.hotel_name.nunique()

109

In [ ]:
# 전체루프 돌리기 
tqdm.pandas()

for file in tqdm(csv_files):

    file_path = os.path.join(base_dir, file)

# 추가 결측치 보강 전체루프

In [275]:
base_dir = args.output_price_processed_missing
base_dir

'./raw_data/price_procssed_missing/'

In [276]:
csv_files= [file for file in os.listdir(base_dir) if file.endswith(".csv")]
csv_files

['L7강남_11.csv',
 'L7홍대_6.csv',
 '관광호텔뮤제오_2.csv',
 '그랜드조선제주_3.csv',
 '그랜드플라자청주호텔_3.csv',
 '그랩디오션송도_16.csv',
 '글래드강남코엑스센터_4.csv',
 '글래드여의도_11.csv',
 '글래드호텔마포_2.csv',
 '글로스터제주_1.csv',
 '나인트리프리미어호텔명동2_13.csv',
 '나인트리호텔명동_10.csv',
 '노보텔앰배서더서울동대문_12.csv',
 '노보텔앰배서더서울용산_2.csv',
 '더클래스300_4.csv',
 '라마다앙코르해운대_4.csv',
 '라발스호텔부산_11.csv',
 '라한셀렉트경주_1.csv',
 '롯데리조트속초_4.csv',
 '롯데시티호텔울산_10.csv',
 '롯데호텔부산_4.csv',
 '롯데호텔서울_18.csv',
 '롯데호텔제주_14.csv',
 '마리안느호텔_5.csv',
 '목시서울인사동호텔_3.csv',
 '미포오션사이드호텔_1.csv',
 '밀리오레서울명동_3.csv',
 '베니키아프리미어호텔해운대_18.csv',
 '베스트루이스해밀턴해운대_7.csv',
 '베스트웨스턴군산_1.csv',
 '베스트웨스턴프리미어강남_3.csv',
 '베스트웨스턴해운대호텔_9.csv',
 '베이하운드호텔_4.csv',
 '부산라메르호텔_7.csv',
 '부산비즈니스호텔_4.csv',
 '서면라이온호텔부산_2.csv',
 '서울신라호텔_3.csv',
 '서울프린스호텔_20.csv',
 '세종호텔_4.csv',
 '센텀프리미어호텔_14.csv',
 '센트럴파크호텔부산_5.csv',
 '솔라리아니시테츠부산_3.csv',
 '솔라리아니시테츠호텔서울명동_5.csv',
 '스위스그랜드호텔_5.csv',
 '스위트호텔남원_3.csv',
 '스탠포드인부산_7.csv',
 '시그니엘부산_1.csv',
 '신라스테이마포_6.csv',
 '신라스테이삼성_6.csv',
 '신라스테이서대문_4.csv',
 '신라스테이서부산_9.csv',
 '신라스테이서초_1.csv'

In [277]:
# 공휴일 데이터 로드
holiday=pd.read_csv(args.raw_data+"/holiday_calender_2022_2024.csv")
holiday=holiday.query('date<="2024-03-31"')
holiday.rename(columns={'date':'booking_date'}, inplace=True)
holiday

,booking_date,day_of_week,dateName,high_demand
0,2022-01-01,Saturday,1월1일,Y
1,2022-01-02,Sunday,N,N
2,2022-01-03,Monday,N,N
3,2022-01-04,Tuesday,N,N
4,2022-01-05,Wednesday,N,N
...,...,...,...,...
816,2024-03-27,Wednesday,N,N
817,2024-03-28,Thursday,N,N
818,2024-03-29,Friday,N,Y
819,2024-03-30,Saturday,N,Y


In [245]:
 df = pd.read_csv(file_path)
df

NameError: name 'file_path' is not defined

In [278]:
# 전체루프 돌리기 
tqdm.pandas()

for file in tqdm(csv_files):

    file_path = os.path.join(base_dir, file)
    
    # 호텔별 인스턴스 가져오기 
    df = pd.read_csv(file_path)
    
    #
    processed_result = missing_processor(df,holiday)
    
    # 처리 데이터 아웃풋
    save_hotel_data2(processed_result,args.output_price_processed_missing_date)
    
    # print
    print('\n')

  0%|          | 0/109 [00:00<?, ?it/s]

room_id
7854    821
7855    821
7858    821
7859    821
7860    821
7864    821
9461    821
9462    821
9469    821
9470    821
9474    821
dtype: int64


room_id
13955    821
13956    821
13959    821
13960    821
13961    821
13964    821
dtype: int64


room_id
21072    821
21108    821
dtype: int64


room_id
2575    821
2576    821
2578    821
dtype: int64


room_id
12064    821
12065    821
12066    821
dtype: int64


room_id
6953     821
6959     821
6962     821
6964     821
6968     821
6969     821
6973     821
6977     821
41934    821
41935    821
41936    821
41937    821
41938    821
41939    821
41941    821
41975    821
dtype: int64


room_id
9287     821
9288     821
28827    821
28828    821
dtype: int64


room_id
1991    821
1995    821
1998    821
2165    821
2170    821
2174    821
2177    821
2497    821
2818    821
2874    821
2875    821
dtype: int64


room_id
2065    821
2332    821
dtype: int64


room_id
1545    821
dtype: int64


room_id
1507    821
1510    821



room_id
8125    821
8477    821
8480    821
9008    821
dtype: int64


room_id
8611    821
8620    821
8819    821
8820    821
dtype: int64


room_id
4202    821
4209    821
dtype: int64


room_id
12367    821
12628    821
dtype: int64


room_id
1925    821
1932    821
1933    821
2653    821
2659    821
2663    821
2665    821
dtype: int64


room_id
19640    821
dtype: int64


room_id
6659    821
dtype: int64


room_id
2872    821
3343    821
3344    821
3388    821
dtype: int64


room_id
17443    821
17444    821
17445    821
17446    821
17650    821
dtype: int64


room_id
16263    821
16501    821
16502    821
dtype: int64


room_id
16345    821
16346    821
dtype: int64


room_id
7428    821
7431    821
7432    821
7434    821
7435    821
8557    821
8558    821
8559    821
8560    821
8564    821
dtype: int64


room_id
1035536    821
dtype: int64


room_id
7558    821
7559    821
7560    821
7561    821
8415    821
8576    821
8578    821
8579    821
8580    821
8846    821
dty

# 처리내역 확인

In [279]:
df=pd.read_csv(args.output_price_processed_missing_date+'페어필드바이메리어트부산송도비치_5.csv')
df

,room_id,hotel_id,hotel_name,ota_type,room_name,booking_date,day_of_week,dateName,high_demand,price_0,...,price_22,price_23,price_24,price_25,price_26,price_27,price_28,price_29,price_30,price_31
0,29709,1449,페어필드바이메리어트부산송도비치,AGODA,"디럭스 씨뷰 룸 (킹베드) (Deluxe, Guest room, 1 King, Se...",2022-01-01,Saturday,1월1일,Y,255000,...,255000,255000,255000,255000,255000,255000,255000,255000,255000,255000
1,29709,1449,페어필드바이메리어트부산송도비치,AGODA,"디럭스 씨뷰 룸 (킹베드) (Deluxe, Guest room, 1 King, Se...",2022-01-02,Sunday,N,N,145000,...,145000,145000,145000,145000,145000,145000,145000,145000,145000,145000
2,29709,1449,페어필드바이메리어트부산송도비치,AGODA,"디럭스 씨뷰 룸 (킹베드) (Deluxe, Guest room, 1 King, Se...",2022-01-03,Monday,N,N,145000,...,145000,145000,145000,145000,145000,145000,145000,145000,145000,145000
3,29709,1449,페어필드바이메리어트부산송도비치,AGODA,"디럭스 씨뷰 룸 (킹베드) (Deluxe, Guest room, 1 King, Se...",2022-01-04,Tuesday,N,N,135000,...,135000,135000,135000,135000,135000,135000,135000,135000,135000,135000
4,29709,1449,페어필드바이메리어트부산송도비치,AGODA,"디럭스 씨뷰 룸 (킹베드) (Deluxe, Guest room, 1 King, Se...",2022-01-05,Wednesday,N,N,135000,...,135000,135000,135000,135000,135000,135000,135000,135000,135000,135000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4100,41985,1449,페어필드바이메리어트부산송도비치,BOOKING,"디럭스 게스트룸 - 고층, 트윈침대/싱글침대 2개",2024-03-27,Wednesday,N,N,214500,...,214500,185000,185000,185000,203500,185000,185000,203500,192500,192500
4101,41985,1449,페어필드바이메리어트부산송도비치,BOOKING,"디럭스 게스트룸 - 고층, 트윈침대/싱글침대 2개",2024-03-28,Thursday,N,N,235000,...,258500,258500,200000,200000,200000,220000,200000,200000,220000,200000
4102,41985,1449,페어필드바이메리어트부산송도비치,BOOKING,"디럭스 게스트룸 - 고층, 트윈침대/싱글침대 2개",2024-03-29,Friday,N,Y,335500,...,205000,225500,225500,200000,200000,200000,200000,200000,200000,220000
4103,41985,1449,페어필드바이메리어트부산송도비치,BOOKING,"디럭스 게스트룸 - 고층, 트윈침대/싱글침대 2개",2024-03-30,Saturday,N,Y,379500,...,275000,275000,302500,302500,230000,230000,230000,230000,230000,230000


In [282]:
for i in df['room_id'].unique():
    filtered_df = df.query(f'room_id == {i}')
    print(f'Shape for room_id {i}:', filtered_df.shape)

Shape for room_id 29709: (821, 41)
Shape for room_id 41947: (821, 41)
Shape for room_id 41948: (821, 41)
Shape for room_id 41949: (821, 41)
Shape for room_id 41985: (821, 41)


# 추가정보합치기 

## 부대시설

In [120]:
base_dir = args.output_price_processed_missing_date
base_dir

'./raw_data/price_processed_missing_date/'

In [121]:
csv_files= [file for file in os.listdir(base_dir) if file.endswith(".csv")]
csv_files

['L7강남_11.csv',
 'L7홍대_6.csv',
 'zNex~$hare솔라리아니시테츠부산_3.csv',
 '관광호텔뮤제오_2.csv',
 '그랜드조선제주_3.csv',
 '그랜드플라자청주호텔_3.csv',
 '그랩디오션송도_16.csv',
 '글래드강남코엑스센터_4.csv',
 '글래드여의도_11.csv',
 '글래드호텔마포_2.csv',
 '글로스터제주_1.csv',
 '나인트리프리미어호텔명동2_13.csv',
 '나인트리호텔명동_10.csv',
 '노보텔앰배서더서울동대문_12.csv',
 '노보텔앰배서더서울용산_2.csv',
 '더클래스300_4.csv',
 '라마다앙코르해운대_4.csv',
 '라발스호텔부산_11.csv',
 '라한셀렉트경주_1.csv',
 '롯데리조트속초_4.csv',
 '롯데시티호텔울산_10.csv',
 '롯데호텔부산_4.csv',
 '롯데호텔서울_18.csv',
 '롯데호텔제주_14.csv',
 '마리안느호텔_5.csv',
 '목시서울인사동호텔_3.csv',
 '미포오션사이드호텔_1.csv',
 '밀리오레서울명동_3.csv',
 '베니키아프리미어호텔해운대_18.csv',
 '베스트루이스해밀턴해운대_7.csv',
 '베스트웨스턴군산_1.csv',
 '베스트웨스턴프리미어강남_3.csv',
 '베스트웨스턴해운대호텔_9.csv',
 '베이하운드호텔_4.csv',
 '부산라메르호텔_7.csv',
 '부산비즈니스호텔_4.csv',
 '서면라이온호텔부산_2.csv',
 '서울신라호텔_3.csv',
 '서울프린스호텔_20.csv',
 '세종호텔_4.csv',
 '센텀프리미어호텔_14.csv',
 '센트럴파크호텔부산_5.csv',
 '솔라리아니시테츠부산_3.csv',
 '솔라리아니시테츠호텔서울명동_5.csv',
 '스위스그랜드호텔_5.csv',
 '스위트호텔남원_3.csv',
 '스탠포드인부산_7.csv',
 '시그니엘부산_1.csv',
 '신라스테이마포_6.csv',
 '신라스테이삼성_6.csv',
 '신라스테이서대문_4.csv',
 '신라

In [122]:
data_frames = []

# 각 파일을 읽고 데이터 프레임 리스트에 추가합니다.
for file in tqdm(csv_files):
    if file.endswith('.csv'):  # 확장자가 .csv인 파일만 처리
        file_path = os.path.join(base_dir, file)
        
        # 파일을 읽어 DataFrame으로 변환
        df = pd.read_csv(file_path)
        
        # DataFrame 리스트에 추가
        data_frames.append(df)

# 모든 DataFrame을 하나로 병합
combined_df = pd.concat(data_frames, ignore_index=True)

  0%|          | 0/110 [00:00<?, ?it/s]

In [123]:
combined_df=combined_df.drop('H', axis=1)
combined_df

,room_id,hotel_id,hotel_name,ota_type,room_name,booking_date,day_of_week,dateName,high_demand,price_0,...,price_22,price_23,price_24,price_25,price_26,price_27,price_28,price_29,price_30,price_31
0,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-01,Saturday,1월1일,Y,123200,...,123200,123200,123200,123200,123200,123200,123200,123200,123200,123200
1,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-02,Sunday,N,N,92400,...,92400,92400,92400,92400,92400,92400,92400,92400,92400,92400
2,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-03,Monday,N,N,92400,...,92400,92400,92400,92400,92400,92400,92400,92400,92400,92400
3,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-04,Tuesday,N,N,92400,...,92400,92400,92400,92400,92400,92400,92400,92400,92400,92400
4,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-05,Wednesday,N,N,88000,...,92400,92400,92400,92400,92400,92400,92400,92400,92400,92400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490132,42358,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,2024-03-27,Wednesday,N,N,394460,...,329120,325006,318835,321920,321920,317806,317806,319864,316778,354046
490133,42358,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,2024-03-28,Thursday,N,N,595320,...,461890,468629,401658,417005,369585,367502,315059,291134,325006,325006
490134,42358,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,2024-03-29,Friday,N,Y,547792,...,547792,547792,547792,547792,547792,547792,548080,547849,547561,548138
490135,42358,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,2024-03-30,Saturday,N,Y,547792,...,547792,547792,547792,547792,547792,547792,548080,547849,547561,548138


In [124]:
for room_id in tqdm(room_ids):
    print(combined_df[combined_df.room_id==room_id].shape[0])

  0%|          | 0/562 [00:00<?, ?it/s]

821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821


In [125]:
current_directory = os.getcwd()
current_directory

'C:\\Users\\user\\Work\\2024_가격예측모델'

In [28]:

# 변경하고자 하는 디렉토리 경로
new_directory = r"C:\Users\user\Work\2024_가격예측모델"

# 현재 디렉토리를 새 경로로 변경
os.chdir(new_directory)

In [126]:
facility=pd.read_csv(args.raw_data+"facility_for_ml.csv")
facility

,hotel_name,road_addr,addr,hotel_id,regino_sigugun,region,lat,lng,hoteLgrade,객실수,...,레스토랑,바,카페,피트니스센터,야외수영장,스파,사우나,연회장,비즈니스센터,해변가
0,아르반호텔,부산 부산진구 중앙대로691번길 32,부산 부산진구 부전동 517-60,101,부산진구,부산,35,129,미등급,94.0,...,0,1,1,0,0,1,1,0,1,0
1,솔라리아니시테츠부산,부산 부산진구 서면로 20,부산 부산진구 부전동 233-16,102,부산진구,부산,35,129,미등급,203.0,...,1,1,1,1,0,1,1,1,1,0
2,부산비즈니스호텔,부산 부산진구 부전로 67,부산 부산진구 부전동 512-4,104,부산진구,부산,35,129,3성,233.0,...,1,1,1,0,0,0,0,1,1,0
3,부산비즈니스호텔,부산 부산진구 부전로 67,부산 부산진구 부전동 512-4,104,부산진구,부산,35,129,3성,233.0,...,1,1,1,0,0,0,0,1,1,0
4,스탠포드인부산,부산 중구 구덕로 53,부산 중구 남포동5가 57-1,105,중구,부산,35,129,3성,132.0,...,1,1,1,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,브라운도트호텔울산삼산,울산 남구 삼산로383번길 6-17,울산 남구 삼산동 216-12,1669,남구,울산,36,129,1성,50.0,...,0,0,0,0,0,0,0,0,0,0
221,켄싱턴리조트서귀포,제주특별자치도 서귀포시 이어도로 684,제주특별자치도 서귀포시 강정동 2677,1674,서귀포시,제주,33,126,미등급,246.0,...,1,1,1,0,1,0,0,1,0,0
222,소노벨비발디파크bc구오크파인,강원 홍천군 서면 한치골길 264,강원 홍천군 서면 팔봉리 1289,1676,홍천군,강원,38,128,미등급,미분류,...,1,1,1,1,1,1,1,1,0,0
223,휘닉스평창호텔,강원 평창군 봉평면 태기로 174,강원 평창군 봉평면 면온리 1095-1,1678,평창군,강원,38,128,4성,130.0,...,1,1,1,1,0,0,1,1,1,0


In [127]:
facility=facility[['hotel_id', 
       'region', 'lat', 'lng', 'hoteLgrade', '객실수', '호텔규모', '업태구분명',
       'review_score', 'number_of_reviews_written', 'index_score', '크롤링_객실수',
       '주차장', '레스토랑', '바', '카페', '피트니스센터', '야외수영장', '스파', '사우나', '연회장',
       '비즈니스센터', '해변가']]
facility

,hotel_id,region,lat,lng,hoteLgrade,객실수,호텔규모,업태구분명,review_score,number_of_reviews_written,...,레스토랑,바,카페,피트니스센터,야외수영장,스파,사우나,연회장,비즈니스센터,해변가
0,101,부산,35,129,미등급,94.0,100개 미만,숙박업 기타,1,0,...,0,1,1,0,0,1,1,0,1,0
1,102,부산,35,129,미등급,203.0,200개 이상,일반호텔,1,0,...,1,1,1,1,0,1,1,1,1,0
2,104,부산,35,129,3성,233.0,200개 이상,관광호텔업,1,0,...,1,1,1,0,0,0,0,1,1,0
3,104,부산,35,129,3성,233.0,200개 이상,관광호텔업,1,0,...,1,1,1,0,0,0,0,1,1,0
4,105,부산,35,129,3성,132.0,200개 미만,관광호텔업,1,0,...,1,1,1,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,1669,울산,36,129,1성,50.0,100개 미만,관광호텔업,1,0,...,0,0,0,0,0,0,0,0,0,0
221,1674,제주,33,126,미등급,246.0,200개 이상,휴양콘도미니엄업,1,0,...,1,1,1,0,1,0,0,1,0,0
222,1676,강원,38,128,미등급,미분류,200개 이상,미분류,1,0,...,1,1,1,1,1,1,1,1,0,0
223,1678,강원,38,128,4성,130.0,200개 미만,관광호텔업,1,0,...,1,1,1,1,0,0,1,1,1,0


In [128]:
combined_df_processed=pd.merge(combined_df,facility, how ='left', on='hotel_id')
combined_df_processed

,room_id,hotel_id,hotel_name,ota_type,room_name,booking_date,day_of_week,dateName,high_demand,price_0,...,레스토랑,바,카페,피트니스센터,야외수영장,스파,사우나,연회장,비즈니스센터,해변가
0,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-01,Saturday,1월1일,Y,123200,...,1,1,0,1,0,0,0,1,1,0
1,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-02,Sunday,N,N,92400,...,1,1,0,1,0,0,0,1,1,0
2,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-03,Monday,N,N,92400,...,1,1,0,1,0,0,0,1,1,0
3,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-04,Tuesday,N,N,92400,...,1,1,0,1,0,0,0,1,1,0
4,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-05,Wednesday,N,N,88000,...,1,1,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505731,42358,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,2024-03-27,Wednesday,N,N,394460,...,1,1,1,1,1,1,1,1,1,0
505732,42358,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,2024-03-28,Thursday,N,N,595320,...,1,1,1,1,1,1,1,1,1,0
505733,42358,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,2024-03-29,Friday,N,Y,547792,...,1,1,1,1,1,1,1,1,1,0
505734,42358,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,2024-03-30,Saturday,N,Y,547792,...,1,1,1,1,1,1,1,1,1,0


In [129]:
combined_df_processed.isnull().sum()

room_id                          0
hotel_id                         0
hotel_name                       0
ota_type                         0
room_name                        0
booking_date                     0
day_of_week                      0
dateName                         0
high_demand                      0
price_0                          0
price_1                          0
price_2                          0
price_3                          0
price_4                          0
price_5                          0
price_6                          0
price_7                          0
price_8                          0
price_9                          0
price_10                         0
price_11                         0
price_12                         0
price_13                         0
price_14                         0
price_15                         0
price_16                         0
price_17                         0
price_18                         0
price_19            

In [130]:
combined_df_processed=combined_df_processed.dropna()
combined_df_processed

,room_id,hotel_id,hotel_name,ota_type,room_name,booking_date,day_of_week,dateName,high_demand,price_0,...,레스토랑,바,카페,피트니스센터,야외수영장,스파,사우나,연회장,비즈니스센터,해변가
0,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-01,Saturday,1월1일,Y,123200,...,1,1,0,1,0,0,0,1,1,0
1,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-02,Sunday,N,N,92400,...,1,1,0,1,0,0,0,1,1,0
2,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-03,Monday,N,N,92400,...,1,1,0,1,0,0,0,1,1,0
3,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-04,Tuesday,N,N,92400,...,1,1,0,1,0,0,0,1,1,0
4,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-05,Wednesday,N,N,88000,...,1,1,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505731,42358,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,2024-03-27,Wednesday,N,N,394460,...,1,1,1,1,1,1,1,1,1,0
505732,42358,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,2024-03-28,Thursday,N,N,595320,...,1,1,1,1,1,1,1,1,1,0
505733,42358,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,2024-03-29,Friday,N,Y,547792,...,1,1,1,1,1,1,1,1,1,0
505734,42358,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,2024-03-30,Saturday,N,Y,547792,...,1,1,1,1,1,1,1,1,1,0


In [131]:
for room_id in tqdm(room_ids):
    print(combined_df[combined_df.room_id==room_id].shape[0])

  0%|          | 0/562 [00:00<?, ?it/s]

821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821


## 객실 디테일

In [132]:
room_detail=pd.read_csv(args.raw_data+"room_detail_03_240110.csv")
room_detail

,id,region,gugun,호텔명,호텔규모,객실명,객실 면적_2,침대 구성_2,전망_2,욕조_2,ag_url
0,101,부산,부산진구,아르반 호텔,100개 이하,디럭스 트윈 (Deluxe Twin),31m²,"['싱글베드 1개', '더블베드 1개']",시티뷰,Y,https://www.agoda.com/ko-kr/arban-hotel/hotel/...
1,101,부산,부산진구,아르반 호텔,100개 이하,디럭스 (더블) (Deluxe - Double),31m²,['킹베드 1개'],시티뷰,Y,https://www.agoda.com/ko-kr/arban-hotel/hotel/...
2,101,부산,부산진구,아르반 호텔,100개 이하,로얄 스위트 (Royal Suite),86m²,"['킹베드 1개', '싱글베드 1개']",시티뷰,Y,https://www.agoda.com/ko-kr/arban-hotel/hotel/...
3,101,부산,부산진구,아르반 호텔,100개 이하,주니어 스위트 (Junior Suite),45m²,"['킹베드 1개', '싱글베드 1개']",시티뷰,Y,https://www.agoda.com/ko-kr/arban-hotel/hotel/...
4,101,부산,부산진구,아르반 호텔,100개 이하,프리미어 디럭스 (Premier Deluxe),38m²,"['퀸베드 1개', '싱글베드 1개']",시티뷰,Y,https://www.agoda.com/ko-kr/arban-hotel/hotel/...
...,...,...,...,...,...,...,...,...,...,...,...
3521,678,경기,수원시,호텔아르떼,100개 이하,디럭스 더블 (Deluxe Double),27m²,['더블베드 1개'],시티뷰,Y,https://www.agoda.com/ko-kr/hotel-arte/hotel/s...
3522,678,경기,수원시,호텔아르떼,100개 이하,디럭스 트윈 (Deluxe Twin),27m²,['싱글베드 2개'],시티뷰,N,https://www.agoda.com/ko-kr/hotel-arte/hotel/s...
3523,678,경기,수원시,호텔아르떼,100개 이하,트리플룸 (Tripe Room),30m²,['싱글베드 3개'],시티뷰,N,https://www.agoda.com/ko-kr/hotel-arte/hotel/s...
3524,678,경기,수원시,호텔아르떼,100개 이하,디럭스 트윈 - 트리플 공용 (Deluxe Twin - Triple Share),27m²,"['더블베드 1개', '싱글베드 1개']",시티뷰,N,https://www.agoda.com/ko-kr/hotel-arte/hotel/s...


In [133]:
room_detail=room_detail.rename(columns={'id':'hotel_id'})
room_detail=room_detail.drop(['region','gugun','호텔규모','ag_url'], axis=1)
room_detail = room_detail[room_detail['hotel_id'].isin(combined_df_processed['hotel_id'])]
room_detail =room_detail.rename(columns={'객실명' : 'room_name','침대 구성_2':'bed','객실 면적_2':'room_size_m2', '전망_2':'view','욕조_2':'bathtub', '호텔명':'hotel_name'})
room_detail

,hotel_id,hotel_name,room_name,room_size_m2,bed,view,bathtub
7,102,솔라리아 니시테츠 부산,스위트 (Suite),37m²,['퀸베드 1개'],시티뷰,Y
8,102,솔라리아 니시테츠 부산,트리플룸 (Triple),37m²,"['싱글베드 2개', '퀸베드 1개']",시티뷰,Y
9,102,솔라리아 니시테츠 부산,스탠다드 트윈 (Standard Twin),21m²,['싱글베드 2개'],시티뷰,N
10,102,솔라리아 니시테츠 부산,슈페리어 트윈 (Superior Twin),26m²,"['더블베드 1개', '싱글베드 1개']",시티뷰,Y
11,102,솔라리아 니시테츠 부산,슈페리어 더블 (Superior Double),26m²,['더블베드 1개'],시티뷰,Y
...,...,...,...,...,...,...,...
3325,378,코리아나호텔,로열 스튜디오 (Royal Studio),73m²,"['싱글베드 1개', '더블베드 1개', '더블베드 1개']",시티뷰,N
3326,378,코리아나호텔,스탠다드 패밀리 트윈 (Standard Family Twin),36m²,['더블베드 2개'],시티뷰,Y
3327,378,코리아나호텔,프리미어룸 (퀸베드) (Premier Queen Room),28m²,['퀸베드 1개'],아웃도어뷰,Y
3328,378,코리아나호텔,프리미어 패밀리룸 (Premier Family Room),36m²,['더블베드 2개'],시티뷰,Y


In [134]:
from ast import literal_eval # 문자열을 리스트로 전환하는 함수
# 문자열을 리스트로 변환하고, 리스트를 문자열로 결합하는 함수
def parse_and_join_bed(bed_str):
    # 문자열을 안전하게 리스트로 변환
    bed_list = literal_eval(bed_str)
    # 리스트를 쉼표로 구분된 문자열로 결합
    return ', '.join(bed_list)

# 'bed' 칼럼 처리
room_detail['bed'] = room_detail['bed'].apply(parse_and_join_bed)
room_detail

,hotel_id,hotel_name,room_name,room_size_m2,bed,view,bathtub
7,102,솔라리아 니시테츠 부산,스위트 (Suite),37m²,퀸베드 1개,시티뷰,Y
8,102,솔라리아 니시테츠 부산,트리플룸 (Triple),37m²,"싱글베드 2개, 퀸베드 1개",시티뷰,Y
9,102,솔라리아 니시테츠 부산,스탠다드 트윈 (Standard Twin),21m²,싱글베드 2개,시티뷰,N
10,102,솔라리아 니시테츠 부산,슈페리어 트윈 (Superior Twin),26m²,"더블베드 1개, 싱글베드 1개",시티뷰,Y
11,102,솔라리아 니시테츠 부산,슈페리어 더블 (Superior Double),26m²,더블베드 1개,시티뷰,Y
...,...,...,...,...,...,...,...
3325,378,코리아나호텔,로열 스튜디오 (Royal Studio),73m²,"싱글베드 1개, 더블베드 1개, 더블베드 1개",시티뷰,N
3326,378,코리아나호텔,스탠다드 패밀리 트윈 (Standard Family Twin),36m²,더블베드 2개,시티뷰,Y
3327,378,코리아나호텔,프리미어룸 (퀸베드) (Premier Queen Room),28m²,퀸베드 1개,아웃도어뷰,Y
3328,378,코리아나호텔,프리미어 패밀리룸 (Premier Family Room),36m²,더블베드 2개,시티뷰,Y


In [135]:
import re

# 정규화 함수 정의
def normalize_room_name(name):
    # 괄호와 괄호 내용 제거
    name = re.sub(r'\s*\(.*?\)\s*', '', name)
    # 특수 문자 제거
    name = re.sub(r'[^\w\s]', '', name)
    # 여러 공백을 하나의 공백으로 통합
    name = re.sub(r'\s+', ' ', name).strip()
    return name

# 데이터프레임의 객실명 정규화 room_name
room_detail['room_name'] = room_detail['room_name'].apply(normalize_room_name)
room_detail

,hotel_id,hotel_name,room_name,room_size_m2,bed,view,bathtub
7,102,솔라리아 니시테츠 부산,스위트,37m²,퀸베드 1개,시티뷰,Y
8,102,솔라리아 니시테츠 부산,트리플룸,37m²,"싱글베드 2개, 퀸베드 1개",시티뷰,Y
9,102,솔라리아 니시테츠 부산,스탠다드 트윈,21m²,싱글베드 2개,시티뷰,N
10,102,솔라리아 니시테츠 부산,슈페리어 트윈,26m²,"더블베드 1개, 싱글베드 1개",시티뷰,Y
11,102,솔라리아 니시테츠 부산,슈페리어 더블,26m²,더블베드 1개,시티뷰,Y
...,...,...,...,...,...,...,...
3325,378,코리아나호텔,로열 스튜디오,73m²,"싱글베드 1개, 더블베드 1개, 더블베드 1개",시티뷰,N
3326,378,코리아나호텔,스탠다드 패밀리 트윈,36m²,더블베드 2개,시티뷰,Y
3327,378,코리아나호텔,프리미어룸,28m²,퀸베드 1개,아웃도어뷰,Y
3328,378,코리아나호텔,프리미어 패밀리룸,36m²,더블베드 2개,시티뷰,Y


In [136]:
room2=room[room['room_id'].isin(combined_df_processed['room_id'])]
room2=room2[['hotel_name','room_id','room_name']]
room2

,hotel_name,room_id,room_name
462,솔라리아 니시테츠 부산,5950,슈페리어 트윈 (Superior Twin)
465,솔라리아 니시테츠 부산,5947,스탠다드 더블 (Standard Double)
470,솔라리아 니시테츠 부산,5949,스탠다드 트윈 (Standard Twin)
984,부산 비즈니스 호텔,6847,스탠다드 더블룸 (Standard Double Room)
988,부산 비즈니스 호텔,6845,스탠다드 트윈룸 (Standard Twin Room)
...,...,...,...
263586,헨나호텔 서울명동,12367,고층 트윈
263760,헨나호텔 서울명동,12628,"트윈룸 (높은 층, 금연) (High Floor Twin Room - Non-Smo..."
265771,힐튼 가든인 서울 강남,5626,디럭스룸 (킹베드) (King Deluxe Room)
265772,힐튼 가든인 서울 강남,5625,룸 (킹베드) (King Guest Room)


In [137]:
room2['room_name'] = room2['room_name'].apply(normalize_room_name)
room2

,hotel_name,room_id,room_name
462,솔라리아 니시테츠 부산,5950,슈페리어 트윈
465,솔라리아 니시테츠 부산,5947,스탠다드 더블
470,솔라리아 니시테츠 부산,5949,스탠다드 트윈
984,부산 비즈니스 호텔,6847,스탠다드 더블룸
988,부산 비즈니스 호텔,6845,스탠다드 트윈룸
...,...,...,...
263586,헨나호텔 서울명동,12367,고층 트윈
263760,헨나호텔 서울명동,12628,트윈룸
265771,힐튼 가든인 서울 강남,5626,디럭스룸
265772,힐튼 가든인 서울 강남,5625,룸


In [138]:
room_detail

,hotel_id,hotel_name,room_name,room_size_m2,bed,view,bathtub
7,102,솔라리아 니시테츠 부산,스위트,37m²,퀸베드 1개,시티뷰,Y
8,102,솔라리아 니시테츠 부산,트리플룸,37m²,"싱글베드 2개, 퀸베드 1개",시티뷰,Y
9,102,솔라리아 니시테츠 부산,스탠다드 트윈,21m²,싱글베드 2개,시티뷰,N
10,102,솔라리아 니시테츠 부산,슈페리어 트윈,26m²,"더블베드 1개, 싱글베드 1개",시티뷰,Y
11,102,솔라리아 니시테츠 부산,슈페리어 더블,26m²,더블베드 1개,시티뷰,Y
...,...,...,...,...,...,...,...
3325,378,코리아나호텔,로열 스튜디오,73m²,"싱글베드 1개, 더블베드 1개, 더블베드 1개",시티뷰,N
3326,378,코리아나호텔,스탠다드 패밀리 트윈,36m²,더블베드 2개,시티뷰,Y
3327,378,코리아나호텔,프리미어룸,28m²,퀸베드 1개,아웃도어뷰,Y
3328,378,코리아나호텔,프리미어 패밀리룸,36m²,더블베드 2개,시티뷰,Y


In [139]:
room3=pd.merge(room2,room_detail, how='left', on=['hotel_name','room_name'])
room3                                               

,hotel_name,room_id,room_name,hotel_id,room_size_m2,bed,view,bathtub
0,솔라리아 니시테츠 부산,5950,슈페리어 트윈,102,26m²,"더블베드 1개, 싱글베드 1개",시티뷰,Y
1,솔라리아 니시테츠 부산,5947,스탠다드 더블,102,19m²,더블베드 1개,시티뷰,N
2,솔라리아 니시테츠 부산,5949,스탠다드 트윈,102,21m²,싱글베드 2개,시티뷰,N
3,부산 비즈니스 호텔,6847,스탠다드 더블룸,104,24m²,킹베드 1개,시티뷰,N
4,부산 비즈니스 호텔,6845,스탠다드 트윈룸,104,24m²,싱글베드 2개,시티뷰,Y
...,...,...,...,...,...,...,...,...
571,헨나호텔 서울명동,12367,고층 트윈,NaN,NaN,NaN,NaN,NaN
572,헨나호텔 서울명동,12628,트윈룸,1571,16m²,싱글베드 2개,마운틴뷰,Y
573,힐튼 가든인 서울 강남,5626,디럭스룸,NaN,NaN,NaN,NaN,NaN
574,힐튼 가든인 서울 강남,5625,룸,NaN,NaN,NaN,NaN,NaN


In [140]:
room3=room3.dropna()
room3

,hotel_name,room_id,room_name,hotel_id,room_size_m2,bed,view,bathtub
0,솔라리아 니시테츠 부산,5950,슈페리어 트윈,102,26m²,"더블베드 1개, 싱글베드 1개",시티뷰,Y
1,솔라리아 니시테츠 부산,5947,스탠다드 더블,102,19m²,더블베드 1개,시티뷰,N
2,솔라리아 니시테츠 부산,5949,스탠다드 트윈,102,21m²,싱글베드 2개,시티뷰,N
3,부산 비즈니스 호텔,6847,스탠다드 더블룸,104,24m²,킹베드 1개,시티뷰,N
4,부산 비즈니스 호텔,6845,스탠다드 트윈룸,104,24m²,싱글베드 2개,시티뷰,Y
...,...,...,...,...,...,...,...,...
566,센트럴파크 호텔 부산,20136,디럭스 쿼드러플룸,1549,31m²,더블베드 2개,하버뷰,N
568,센트럴파크 호텔 부산,20125,스탠다드 더블 하버뷰,1549,25m²,더블베드 1개,하버뷰,N
569,센트럴파크 호텔 부산,20129,스탠다드 트윈 하버뷰,1549,25m²,싱글베드 2개,하버뷰,N
570,센트럴파크 호텔 부산,20138,하버뷰 패밀리 스위트,1549,45m²,더블베드 2개,하버뷰,Y


In [141]:
for i in range(0,5):
    print(room3.loc[i, 'room_size_m2'])

26m²
19m²
21m²
24m²
24m²


In [142]:
room3['room_size_m2'] = room3['room_size_m2'].str.replace('m²', '', regex=False)
room3

,hotel_name,room_id,room_name,hotel_id,room_size_m2,bed,view,bathtub
0,솔라리아 니시테츠 부산,5950,슈페리어 트윈,102,26,"더블베드 1개, 싱글베드 1개",시티뷰,Y
1,솔라리아 니시테츠 부산,5947,스탠다드 더블,102,19,더블베드 1개,시티뷰,N
2,솔라리아 니시테츠 부산,5949,스탠다드 트윈,102,21,싱글베드 2개,시티뷰,N
3,부산 비즈니스 호텔,6847,스탠다드 더블룸,104,24,킹베드 1개,시티뷰,N
4,부산 비즈니스 호텔,6845,스탠다드 트윈룸,104,24,싱글베드 2개,시티뷰,Y
...,...,...,...,...,...,...,...,...
566,센트럴파크 호텔 부산,20136,디럭스 쿼드러플룸,1549,31,더블베드 2개,하버뷰,N
568,센트럴파크 호텔 부산,20125,스탠다드 더블 하버뷰,1549,25,더블베드 1개,하버뷰,N
569,센트럴파크 호텔 부산,20129,스탠다드 트윈 하버뷰,1549,25,싱글베드 2개,하버뷰,N
570,센트럴파크 호텔 부산,20138,하버뷰 패밀리 스위트,1549,45,더블베드 2개,하버뷰,Y


In [143]:
room3=room3[[ 'room_id', 'room_name', 'hotel_id', 'room_size_m2', 'bed',
       'view', 'bathtub']]
room3

,room_id,room_name,hotel_id,room_size_m2,bed,view,bathtub
0,5950,슈페리어 트윈,102,26,"더블베드 1개, 싱글베드 1개",시티뷰,Y
1,5947,스탠다드 더블,102,19,더블베드 1개,시티뷰,N
2,5949,스탠다드 트윈,102,21,싱글베드 2개,시티뷰,N
3,6847,스탠다드 더블룸,104,24,킹베드 1개,시티뷰,N
4,6845,스탠다드 트윈룸,104,24,싱글베드 2개,시티뷰,Y
...,...,...,...,...,...,...,...
566,20136,디럭스 쿼드러플룸,1549,31,더블베드 2개,하버뷰,N
568,20125,스탠다드 더블 하버뷰,1549,25,더블베드 1개,하버뷰,N
569,20129,스탠다드 트윈 하버뷰,1549,25,싱글베드 2개,하버뷰,N
570,20138,하버뷰 패밀리 스위트,1549,45,더블베드 2개,하버뷰,Y


In [144]:
combined_df_processed=combined_df_processed.drop(['객실수','호텔규모'],axis=1)
combined_df_processed

,room_id,hotel_id,hotel_name,ota_type,room_name,booking_date,day_of_week,dateName,high_demand,price_0,...,레스토랑,바,카페,피트니스센터,야외수영장,스파,사우나,연회장,비즈니스센터,해변가
0,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-01,Saturday,1월1일,Y,123200,...,1,1,0,1,0,0,0,1,1,0
1,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-02,Sunday,N,N,92400,...,1,1,0,1,0,0,0,1,1,0
2,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-03,Monday,N,N,92400,...,1,1,0,1,0,0,0,1,1,0
3,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-04,Tuesday,N,N,92400,...,1,1,0,1,0,0,0,1,1,0
4,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-05,Wednesday,N,N,88000,...,1,1,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505731,42358,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,2024-03-27,Wednesday,N,N,394460,...,1,1,1,1,1,1,1,1,1,0
505732,42358,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,2024-03-28,Thursday,N,N,595320,...,1,1,1,1,1,1,1,1,1,0
505733,42358,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,2024-03-29,Friday,N,Y,547792,...,1,1,1,1,1,1,1,1,1,0
505734,42358,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,2024-03-30,Saturday,N,Y,547792,...,1,1,1,1,1,1,1,1,1,0


In [147]:
combined_df_processed=pd.merge(combined_df_processed,room3, how='left', on=['hotel_id','room_id'])
combined_df_processed

,room_id,hotel_id,hotel_name,ota_type,room_name_x,booking_date,day_of_week,dateName,high_demand,price_0,...,spa,sauna,banquet_hall,business_center,beachfront,room_name_y,room_size_m2,bed,view,bathtub
0,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-01,Saturday,1월1일,Y,123200,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
1,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-02,Sunday,N,N,92400,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
2,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-03,Monday,N,N,92400,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
3,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-04,Tuesday,N,N,92400,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
4,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-05,Wednesday,N,N,88000,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488490,42358,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,2024-03-27,Wednesday,N,N,394460,...,1,1,1,1,0,NaN,NaN,NaN,NaN,NaN
488491,42358,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,2024-03-28,Thursday,N,N,595320,...,1,1,1,1,0,NaN,NaN,NaN,NaN,NaN
488492,42358,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,2024-03-29,Friday,N,Y,547792,...,1,1,1,1,0,NaN,NaN,NaN,NaN,NaN
488493,42358,1309,힐튼호텔경주,BOOKING,프리미엄 킹룸 - 호수 전망,2024-03-30,Saturday,N,Y,547792,...,1,1,1,1,0,NaN,NaN,NaN,NaN,NaN


In [148]:
combined_df_processed=combined_df_processed.dropna()
combined_df_processed

,room_id,hotel_id,hotel_name,ota_type,room_name_x,booking_date,day_of_week,dateName,high_demand,price_0,...,spa,sauna,banquet_hall,business_center,beachfront,room_name_y,room_size_m2,bed,view,bathtub
0,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-01,Saturday,1월1일,Y,123200,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
1,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-02,Sunday,N,N,92400,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
2,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-03,Monday,N,N,92400,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
3,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-04,Tuesday,N,N,92400,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
4,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-05,Wednesday,N,N,88000,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483564,41962,1309,힐튼호텔경주,BOOKING,디럭스 패밀리룸,2024-03-27,Wednesday,N,N,297660,...,1,1,1,1,0,디럭스 패밀리룸,38,"싱글베드 1개, 킹베드 1개",마운틴뷰,Y
483565,41962,1309,힐튼호텔경주,BOOKING,디럭스 패밀리룸,2024-03-28,Thursday,N,N,544500,...,1,1,1,1,0,디럭스 패밀리룸,38,"싱글베드 1개, 킹베드 1개",마운틴뷰,Y
483566,41962,1309,힐튼호텔경주,BOOKING,디럭스 패밀리룸,2024-03-29,Friday,N,Y,389160,...,1,1,1,1,0,디럭스 패밀리룸,38,"싱글베드 1개, 킹베드 1개",마운틴뷰,Y
483567,41962,1309,힐튼호텔경주,BOOKING,디럭스 패밀리룸,2024-03-30,Saturday,N,Y,389160,...,1,1,1,1,0,디럭스 패밀리룸,38,"싱글베드 1개, 킹베드 1개",마운틴뷰,Y


In [149]:
combined_df_processed.rename(columns={
    '크롤링_객실수': 'total_rooms',
    '주차장': 'parking',
    '레스토랑': 'restaurant',
    '바': 'bar',
    '카페': 'cafe',
    '피트니스센터': 'fitness_center',
    '야외수영장': 'outdoor_pool',
    '스파': 'spa',
    '사우나': 'sauna',
    '연회장': 'banquet_hall',
    '비즈니스센터': 'business_center',
    '해변가': 'beachfront',
    '업태구분명': 'business_type'
}, inplace=True)

In [150]:
combined_df_processed.room_id.nunique()

278

In [151]:
combined_df_processed.hotel_id.nunique()

85

In [152]:
combined_df_processed.total_rooms.min()

28.0

In [153]:
combined_df_processed

,room_id,hotel_id,hotel_name,ota_type,room_name_x,booking_date,day_of_week,dateName,high_demand,price_0,...,spa,sauna,banquet_hall,business_center,beachfront,room_name_y,room_size_m2,bed,view,bathtub
0,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-01,Saturday,1월1일,Y,123200,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
1,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-02,Sunday,N,N,92400,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
2,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-03,Monday,N,N,92400,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
3,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-04,Tuesday,N,N,92400,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
4,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-05,Wednesday,N,N,88000,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483564,41962,1309,힐튼호텔경주,BOOKING,디럭스 패밀리룸,2024-03-27,Wednesday,N,N,297660,...,1,1,1,1,0,디럭스 패밀리룸,38,"싱글베드 1개, 킹베드 1개",마운틴뷰,Y
483565,41962,1309,힐튼호텔경주,BOOKING,디럭스 패밀리룸,2024-03-28,Thursday,N,N,544500,...,1,1,1,1,0,디럭스 패밀리룸,38,"싱글베드 1개, 킹베드 1개",마운틴뷰,Y
483566,41962,1309,힐튼호텔경주,BOOKING,디럭스 패밀리룸,2024-03-29,Friday,N,Y,389160,...,1,1,1,1,0,디럭스 패밀리룸,38,"싱글베드 1개, 킹베드 1개",마운틴뷰,Y
483567,41962,1309,힐튼호텔경주,BOOKING,디럭스 패밀리룸,2024-03-30,Saturday,N,Y,389160,...,1,1,1,1,0,디럭스 패밀리룸,38,"싱글베드 1개, 킹베드 1개",마운틴뷰,Y


In [154]:
combined_df_processed=combined_df_processed.drop_duplicates()
combined_df_processed

,room_id,hotel_id,hotel_name,ota_type,room_name_x,booking_date,day_of_week,dateName,high_demand,price_0,...,spa,sauna,banquet_hall,business_center,beachfront,room_name_y,room_size_m2,bed,view,bathtub
0,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-01,Saturday,1월1일,Y,123200,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
1,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-02,Sunday,N,N,92400,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
2,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-03,Monday,N,N,92400,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
3,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-04,Tuesday,N,N,92400,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
4,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-05,Wednesday,N,N,88000,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483564,41962,1309,힐튼호텔경주,BOOKING,디럭스 패밀리룸,2024-03-27,Wednesday,N,N,297660,...,1,1,1,1,0,디럭스 패밀리룸,38,"싱글베드 1개, 킹베드 1개",마운틴뷰,Y
483565,41962,1309,힐튼호텔경주,BOOKING,디럭스 패밀리룸,2024-03-28,Thursday,N,N,544500,...,1,1,1,1,0,디럭스 패밀리룸,38,"싱글베드 1개, 킹베드 1개",마운틴뷰,Y
483566,41962,1309,힐튼호텔경주,BOOKING,디럭스 패밀리룸,2024-03-29,Friday,N,Y,389160,...,1,1,1,1,0,디럭스 패밀리룸,38,"싱글베드 1개, 킹베드 1개",마운틴뷰,Y
483567,41962,1309,힐튼호텔경주,BOOKING,디럭스 패밀리룸,2024-03-30,Saturday,N,Y,389160,...,1,1,1,1,0,디럭스 패밀리룸,38,"싱글베드 1개, 킹베드 1개",마운틴뷰,Y


In [155]:
from src.utils.data_utils import reduce_memory_footprint

In [156]:
combined_df_processed.info(memory_usage="deep", verbose=False)

<class 'pandas.core.frame.DataFrame'>
Index: 238911 entries, 0 to 483568
Columns: 66 entries, room_id to bathtub
dtypes: float64(51), object(15)
memory usage: 407.2 MB


In [157]:
combined_df_processed = reduce_memory_footprint(combined_df_processed)

In [158]:
combined_df_processed.info(memory_usage="deep", verbose=False)

<class 'pandas.core.frame.DataFrame'>
Index: 238911 entries, 0 to 483568
Columns: 66 entries, room_id to bathtub
dtypes: category(15), float32(51)
memory usage: 52.6 MB


In [159]:
combined_df_processed['booking_date']=pd.to_datetime(combined_df_processed['booking_date'])

In [160]:
room_ids = sorted(combined_df_processed.room_id.unique())

In [172]:
for room_id in tqdm(room_ids):
    print(combined_df_processed[combined_df_processed.room_id==room_id].shape[0])

  0%|          | 0/278 [00:00<?, ?it/s]

821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821
821


In [170]:
combined_df_processed.drop_duplicates(subset=['booking_date', 'room_id'], inplace=True)


In [167]:

combined_df_processed.drop_duplicates(inplace=True)


In [171]:
combined_df_processed.query('room_id==6845').head(4).T

,179799,179801,179803,179805
room_id,6845,6845,6845,6845
hotel_id,104,104,104,104
hotel_name,부산비즈니스호텔,부산비즈니스호텔,부산비즈니스호텔,부산비즈니스호텔
ota_type,AGODA,AGODA,AGODA,AGODA
room_name_x,스탠다드 트윈룸 (Standard Twin Room),스탠다드 트윈룸 (Standard Twin Room),스탠다드 트윈룸 (Standard Twin Room),스탠다드 트윈룸 (Standard Twin Room)
booking_date,2022-01-01 00:00:00,2022-01-02 00:00:00,2022-01-03 00:00:00,2022-01-04 00:00:00
day_of_week,Saturday,Sunday,Monday,Tuesday
dateName,1월1일,N,N,N
high_demand,Y,N,N,N
price_0,77488,64573,64573,64838


In [173]:
test_mask = (combined_df_processed.booking_date.dt.year==2024) & (combined_df_processed.booking_date.dt.month==2)
val_mask = (combined_df_processed.booking_date.dt.year==2024) & (combined_df_processed.booking_date.dt.month==1)

In [174]:
train = combined_df_processed[~(val_mask|test_mask)]
val = combined_df_processed[val_mask]
test = combined_df_processed[test_mask]

In [175]:
addtional_mask = (combined_df_processed.booking_date.dt.year==2024) & (combined_df_processed.booking_date.dt.month==3)

In [176]:
train = combined_df_processed[~(val_mask|test_mask|addtional_mask)]
train

,room_id,hotel_id,hotel_name,ota_type,room_name_x,booking_date,day_of_week,dateName,high_demand,price_0,...,spa,sauna,banquet_hall,business_center,beachfront,room_name_y,room_size_m2,bed,view,bathtub
0,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-01,Saturday,1월1일,Y,123200,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
1,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-02,Sunday,N,N,92400,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
2,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-03,Monday,N,N,92400,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
3,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-04,Tuesday,N,N,92400,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
4,7854,1378,L7강남,AGODA,스탠다드 트윈 (Standard Twin),2022-01-05,Wednesday,N,N,88000,...,0,0,1,1,0,스탠다드 트윈,26,싱글베드 2개,시티뷰,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483473,41962,1309,힐튼호텔경주,BOOKING,디럭스 패밀리룸,2023-12-27,Wednesday,N,N,273460,...,1,1,1,1,0,디럭스 패밀리룸,38,"싱글베드 1개, 킹베드 1개",마운틴뷰,Y
483474,41962,1309,힐튼호텔경주,BOOKING,디럭스 패밀리룸,2023-12-28,Thursday,N,N,261360,...,1,1,1,1,0,디럭스 패밀리룸,38,"싱글베드 1개, 킹베드 1개",마운틴뷰,Y
483475,41962,1309,힐튼호텔경주,BOOKING,디럭스 패밀리룸,2023-12-29,Friday,N,Y,304460,...,1,1,1,1,0,디럭스 패밀리룸,38,"싱글베드 1개, 킹베드 1개",마운틴뷰,Y
483476,41962,1309,힐튼호텔경주,BOOKING,디럭스 패밀리룸,2023-12-30,Saturday,N,Y,479160,...,1,1,1,1,0,디럭스 패밀리룸,38,"싱글베드 1개, 킹베드 1개",마운틴뷰,Y


In [179]:
train.query('room_id==6845')

,room_id,hotel_id,hotel_name,ota_type,room_name_x,booking_date,day_of_week,dateName,high_demand,price_0,...,spa,sauna,banquet_hall,business_center,beachfront,room_name_y,room_size_m2,bed,view,bathtub
179799,6845,104,부산비즈니스호텔,AGODA,스탠다드 트윈룸 (Standard Twin Room),2022-01-01,Saturday,1월1일,Y,77488,...,0,0,1,1,0,스탠다드 트윈룸,24,싱글베드 2개,시티뷰,Y
179801,6845,104,부산비즈니스호텔,AGODA,스탠다드 트윈룸 (Standard Twin Room),2022-01-02,Sunday,N,N,64573,...,0,0,1,1,0,스탠다드 트윈룸,24,싱글베드 2개,시티뷰,Y
179803,6845,104,부산비즈니스호텔,AGODA,스탠다드 트윈룸 (Standard Twin Room),2022-01-03,Monday,N,N,64573,...,0,0,1,1,0,스탠다드 트윈룸,24,싱글베드 2개,시티뷰,Y
179805,6845,104,부산비즈니스호텔,AGODA,스탠다드 트윈룸 (Standard Twin Room),2022-01-04,Tuesday,N,N,64838,...,0,0,1,1,0,스탠다드 트윈룸,24,싱글베드 2개,시티뷰,Y
179807,6845,104,부산비즈니스호텔,AGODA,스탠다드 트윈룸 (Standard Twin Room),2022-01-05,Wednesday,N,N,64838,...,0,0,1,1,0,스탠다드 트윈룸,24,싱글베드 2개,시티뷰,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181249,6845,104,부산비즈니스호텔,AGODA,스탠다드 트윈룸 (Standard Twin Room),2023-12-27,Wednesday,N,N,95830,...,0,0,1,1,0,스탠다드 트윈룸,24,싱글베드 2개,시티뷰,Y
181251,6845,104,부산비즈니스호텔,AGODA,스탠다드 트윈룸 (Standard Twin Room),2023-12-28,Thursday,N,N,95830,...,0,0,1,1,0,스탠다드 트윈룸,24,싱글베드 2개,시티뷰,Y
181253,6845,104,부산비즈니스호텔,AGODA,스탠다드 트윈룸 (Standard Twin Room),2023-12-29,Friday,N,Y,136110,...,0,0,1,1,0,스탠다드 트윈룸,24,싱글베드 2개,시티뷰,Y
181255,6845,104,부산비즈니스호텔,AGODA,스탠다드 트윈룸 (Standard Twin Room),2023-12-30,Saturday,N,Y,126360,...,0,0,1,1,0,스탠다드 트윈룸,24,싱글베드 2개,시티뷰,Y


In [177]:
print(f"# of Training samples: {len(train)} | # of Validation samples: {len(val)} | # of Test samples: {len(test)}")
print(f"Max Date in Train: {train.booking_date.max()} | Min Date in Validation: {val.booking_date.min()} | Min Date in Test: {test.booking_date.min()}")
print(f"Min Date in Train: {train.booking_date.min()} | Max Date in Validation: {val.booking_date.max()} | Max Date in Test: {test.booking_date.max()}")

# of Training samples: 202940 | # of Validation samples: 8618 | # of Test samples: 8062
Max Date in Train: 2023-12-31 00:00:00 | Min Date in Validation: 2024-01-01 00:00:00 | Min Date in Test: 2024-02-01 00:00:00
Min Date in Train: 2022-01-01 00:00:00 | Max Date in Validation: 2024-01-31 00:00:00 | Max Date in Test: 2024-02-29 00:00:00


In [178]:
train.to_parquet("./raw_data/price_processesd_missing_date_additional/selected_blocks_train_missing_imputed.parquet")
val.to_parquet("./raw_data/price_processesd_missing_date_additional/selected_blocks_val_missing_imputed.parquet")
test.to_parquet("./raw_data/price_processesd_missing_date_additional/selected_blocks_test_missing_imputed.parquet")

# 시각화

In [84]:
m_df= df[["price_0"]].resample("MS").mean()
w_df=df[['price_0']].resample("W-MON").mean()
display(w_df.head())

,price_0
booking_date,
2022-01-03,167300
2022-01-10,131343
2022-01-17,137500
2022-01-24,138286
2022-01-31,173643


In [90]:
df['2023-05-27':]

,room_id,hotel_id,hotel_name,ota_type,room_name,day_of_week,dateName,high_demand,price_0,price_1,...,price_20,price_21,price_22,price_23,price_24,price_25,price_26,price_27,price_28,price_29
booking_date,,,,,,,,,,,,,,,,,,,,,
2023-05-27,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Saturday,부처님오신날,Y,528000,528000,...,390500,313500,313500,313500,313500,313500,313500,313500,313500,313500
2023-05-28,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Sunday,N,Y,412500,423500,...,368500,291500,269500,269500,269500,258500,258500,258500,258500,258500
2023-05-29,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Monday,대체공휴일,N,176000,176000,...,181500,181500,181500,170500,170500,170500,170500,170500,170500,170500
2023-05-30,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Tuesday,N,N,187000,187000,...,192500,192500,192500,170500,170500,170500,170500,170500,170500,170500
2023-05-31,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Wednesday,N,N,280500,280500,...,280500,280500,280500,280500,225500,214500,214500,203500,170500,170500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-27,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Wednesday,N,N,165000,165000,...,181500,165000,181500,155000,155000,155000,170500,155000,155000,170500
2024-03-28,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Thursday,N,N,245000,245000,...,205000,225500,225500,225500,170000,170000,170000,187000,170000,170000
2024-03-29,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Friday,N,Y,302500,275000,...,192500,175000,175000,192500,192500,170000,170000,170000,170000,170000


In [102]:
df

,room_id,hotel_id,hotel_name,ota_type,room_name,day_of_week,dateName,high_demand,price_0,price_1,...,price_20,price_21,price_22,price_23,price_24,price_25,price_26,price_27,price_28,price_29
booking_date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Saturday,1월1일,Y,247500,247500,...,247500,247500,247500,247500,247500,247500,247500,247500,247500,247500
2022-01-02,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Sunday,N,N,127200,126500,...,126500,126500,126500,126500,126500,126500,126500,126500,126500,126500
2022-01-03,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Monday,N,N,127200,127200,...,126500,126500,126500,126500,126500,126500,126500,126500,126500,126500
2022-01-04,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Tuesday,N,N,116400,116400,...,115500,115500,115500,115500,115500,115500,115500,115500,115500,115500
2022-01-05,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Wednesday,N,N,115500,116400,...,115500,115500,115500,115500,115500,115500,115500,115500,115500,115500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-27,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Wednesday,N,N,165000,165000,...,181500,165000,181500,155000,155000,155000,170500,155000,155000,170500
2024-03-28,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Thursday,N,N,245000,245000,...,205000,225500,225500,225500,170000,170000,170000,187000,170000,170000
2024-03-29,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Friday,N,Y,302500,275000,...,192500,175000,175000,192500,192500,170000,170000,170000,170000,170000


In [95]:
hotel_name=df.hotel_name.iloc[0]
room_name=df.room_name.iloc[0]

In [99]:
import plotly.graph_objects as go

# 데이터의 일 변화, 주변화, 월별화 추이를 파악할때 용이하겠다. 
fig = go.Figure()

fig.update_layout(title=f"{hotel_name}_{room_name}")

fig.add_trace(go.Scatter(
    x=df.index,
    y=df.price_0,
    opacity=0.4,
    name="당일가",
    line={"dash": "dot"}
))

fig.add_trace(go.Scatter(
    x=m_df.index,
    y=m_df['price_0'],
    opacity=1.0,
    name=" 월간평균",
#     line={"dash": "dashdot"}
))
fig.add_trace(go.Scatter(
    x=w_df.index,
    y=w_df['price_0'],
    opacity=1,
    name="주간평균",
    line={"dash": "dash"},
))
fig.show()


In [152]:
df

,room_id,hotel_id,hotel_name,ota_type,room_name,day_of_week,dateName,high_demand,price_0,price_1,...,price_20,price_21,price_22,price_23,price_24,price_25,price_26,price_27,price_28,price_29
booking_date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Saturday,1월1일,Y,247500,247500,...,247500,247500,247500,247500,247500,247500,247500,247500,247500,247500
2022-01-02,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Sunday,N,N,127200,126500,...,126500,126500,126500,126500,126500,126500,126500,126500,126500,126500
2022-01-03,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Monday,N,N,127200,127200,...,126500,126500,126500,126500,126500,126500,126500,126500,126500,126500
2022-01-04,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Tuesday,N,N,116400,116400,...,115500,115500,115500,115500,115500,115500,115500,115500,115500,115500
2022-01-05,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Wednesday,N,N,115500,116400,...,115500,115500,115500,115500,115500,115500,115500,115500,115500,115500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-27,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Wednesday,N,N,165000,165000,...,181500,165000,181500,155000,155000,155000,170500,155000,155000,170500
2024-03-28,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Thursday,N,N,245000,245000,...,205000,225500,225500,225500,170000,170000,170000,187000,170000,170000
2024-03-29,41947,1449,페어필드바이메리어트부산송도비치,BOOKING,"스탠다드룸 - 게스트룸, 킹침대 1개, 바다 전망",Friday,N,Y,302500,275000,...,192500,175000,175000,192500,192500,170000,170000,170000,170000,170000


In [100]:
m_df= df[["price_29"]].resample("MS").mean()
w_df=df[['price_29']].resample("W-MON").mean()

In [104]:
df.columns

Index(['room_id', 'hotel_id', 'hotel_name', 'ota_type', 'room_name',
       'day_of_week', 'dateName', 'high_demand', 'price_0', 'price_1',
       'price_2', 'price_3', 'price_4', 'price_5', 'price_6', 'price_7',
       'price_8', 'price_9', 'price_10', 'price_11', 'price_12', 'price_13',
       'price_14', 'price_15', 'price_16', 'price_17', 'price_18', 'price_19',
       'price_20', 'price_21', 'price_22', 'price_23', 'price_24', 'price_25',
       'price_26', 'price_27', 'price_28', 'price_29'],
      dtype='object')

In [113]:
w_df=df[['price_0', 'price_1',
       'price_2', 'price_3', 'price_4', 'price_5', 'price_6', 'price_7',
       'price_8', 'price_9', 'price_10', 'price_11', 'price_12', 'price_13',
       'price_14', 'price_15', 'price_16', 'price_17', 'price_18', 'price_19',
       'price_20', 'price_21', 'price_22', 'price_23', 'price_24', 'price_25',
       'price_26', 'price_27', 'price_28', 'price_29']].resample("W-MON").mean()

In [111]:
m_df=df[['price_0', 'price_1',
       'price_2', 'price_3', 'price_4', 'price_5', 'price_6', 'price_7',
       'price_8', 'price_9', 'price_10', 'price_11', 'price_12', 'price_13',
       'price_14', 'price_15', 'price_16', 'price_17', 'price_18', 'price_19',
       'price_20', 'price_21', 'price_22', 'price_23', 'price_24', 'price_25',
       'price_26', 'price_27', 'price_28', 'price_29']].resample("MS").mean()
m_df

,price_0,price_1,price_2,price_3,price_4,price_5,price_6,price_7,price_8,price_9,...,price_20,price_21,price_22,price_23,price_24,price_25,price_26,price_27,price_28,price_29
booking_date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,147332,147339,147345,147365,147397,148448,148790,148942,148942,148942,...,148611,148585,148585,148585,148598,148618,148676,148727,148647,148566
2022-02-01,145750,145750,145750,146143,145161,145161,145161,145161,145161,145554,...,145554,145554,145946,145946,145946,143373,143786,143786,143839,143857
2022-03-01,138210,138210,138210,138565,137323,137323,137855,137855,137323,136790,...,132887,132887,132887,133597,133597,133597,134484,134484,134661,134661
2022-04-01,154183,153450,153450,153450,153450,153450,153083,153083,153083,152533,...,149472,149380,148262,148097,147620,148115,148665,148848,148298,147895
2022-05-01,169258,169258,170039,169613,168726,166774,166774,166419,166242,164823,...,156448,157300,157956,158240,158861,158968,159465,161115,161984,162534
2022-06-01,175708,175078,173975,173888,172558,172198,169282,168635,167235,165198,...,146275,146480,146480,145365,145365,144540,144045,144045,143880,143880
2022-07-01,230113,230113,231177,230113,229226,230468,234903,235613,235968,236677,...,204068,203163,201029,200090,200032,198702,198063,196456,195450,194434
2022-08-01,254082,254082,255005,253905,253905,254260,257453,258695,260115,260115,...,239356,239676,240155,237760,237760,238079,238398,238398,239037,239516
2022-09-01,189567,188833,188467,186817,185350,186083,187367,187367,185717,184617,...,164010,163515,163185,161865,162360,163680,163845,163845,163680,163185


In [148]:
pio.templates.default = "plotly_white"
pio.kaleido.scope.mathjax = None

In [151]:
# 데이터의 일 변화, 주변화, 월별화 추이를 파악할때 용이하겠다. 
fig = go.Figure()

fig.update_layout(title=f"{hotel_name}_{room_name}")

fig.add_trace(go.Scatter(
    x=m_df.index,
    y=m_df['price_0'],
    opacity=0.4,
    name="price_0",
    #line={"dash": "dot"}
))


fig.add_trace(go.Scatter(
    x=m_df.index,
    y=m_df['price_29'],
    opacity=1,
    name="price_29",
 #   line={"dash": "dash"},
))

fig.show()


In [115]:
# 데이터의 일 변화, 주변화, 월별화 추이를 파악할때 용이하겠다. 
fig = go.Figure()

fig.update_layout(title=f"{hotel_name}_{room_name}")

fig.add_trace(go.Scatter(
    x=m_df.index,
    y=m_df['price_0'],
    opacity=0.4,
    name="price_0",
    line={"dash": "dot"}
))

fig.add_trace(go.Scatter(
    x=m_df.index,
    y=m_df['price_6'],
    opacity=1.0,
    name=" price_6",
   line={"dash": "dashdot"}
))

fig.add_trace(go.Scatter(
    x=m_df.index,
    y=m_df['price_14'],
    opacity=1,
    name="price_14",
    line={"dash": "dash"},
))


fig.add_trace(go.Scatter(
    x=m_df.index,
    y=m_df['price_29'],
    opacity=1,
    name="price_29",
    line={"dash": "dash"},
))

fig.show()


In [119]:
# 데이터의 일 변화, 주변화, 월별화 추이를 파악할때 용이하겠다. 
fig = go.Figure()

fig.update_layout(title=f"{hotel_name}_{room_name}")

fig.add_trace(go.Scatter(
    x=w_df.index,
    y=w_df['price_0'],
    opacity=0.4,
    name="price_0", 
    line=dict(color='black')
))

fig.add_trace(go.Scatter(
    x=w_df.index,
    y=w_df['price_6'],
    opacity=1.0,
    name=" price_6",
   line={"dash": "dashdot"}
))

fig.add_trace(go.Scatter(
    x=w_df.index,
    y=w_df['price_14'],
    opacity=1,
    name="price_14",
    line={"dash": "dash"},
))


fig.add_trace(go.Scatter(
    x=w_df.index,
    y=w_df['price_29'],
    opacity=1,
    name="price_29",
    line={"dash": "dash"},
))

fig.show()

In [171]:
# 데이터의 일 변화, 주변화, 월별화 추이를 파악할때 용이하겠다. 
fig = go.Figure()

fig.update_layout(title=f"{hotel_name}_{room_name}")

fig.add_trace(go.Scatter(
    x=df_target.index,
    y=df_target['price_0'],
    opacity=0.4,
    name="price_0", 
    line=dict(color='black')
))

fig.add_trace(go.Scatter(
    x=df_target.index,
    y=df_target['price_13'],
    opacity=1,
    name="price_13",
    line={"dash": "dash"},
))


fig.add_trace(go.Scatter(
    x=df_target.index,
    y=df_target['price_29'],
    opacity=1,
    name="price_29",
    line={"dash": "dash"},
))

fig.show()

In [138]:
# 데이터의 일 변화, 주변화, 월별화 추이를 파악할때 용이하겠다. 
fig = go.Figure()

fig.update_layout(title=f"{hotel_name}_{room_name}")

fig.add_trace(go.Scatter(
    x=df_july.index,
    y=df_july['price_0'],
    opacity=0.4,
    name="price_0", 
    line=dict(color='black')
))


fig.add_trace(go.Scatter(
    x=df_july.index,
    y=df_july['price_29'],
    opacity=1,
    name="price_29",
    line={"dash": "dash"},
))

fig.show()